In [ ]:
!pip install chromadb

In [ ]:
!pip install imgkit

In [ ]:
!python -m spacy download en_core_web_sm 

In [ ]:
!pip install -U deep-translator

In [1]:
import torch
import spacy
import PyPDF2
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
import sentence_transformers
import re
import imgkit
from PIL import Image, ImageDraw, ImageFont
from flask import redirect, url_for, request, jsonify, send_file
from gpt4all import GPT4All
from deep_translator import GoogleTranslator

/tmp/ipykernel_49210/1250275816.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
model = GPT4All("ggml-model-gpt4all-falcon-q4_0.bin")
output = model.generate("The capital of France is ", max_tokens=3)
print(output)

Found model file at  /home/diginsight/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin
falcon_model_load: loading model from '/home/diginsight/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: ........................ done
falcon_model_load: model size =  3872.59 MB / num tensors = 196
"Paris".


In [3]:
# Use any translator you like, in this example GoogleTranslator
translated = GoogleTranslator(source='id', target='en').translate("Tolong rangkum kalimat berikut : Homo sapiens pertama diperkirakan masuk ke Nusantara sejak 100.000 tahun yang lalu melewati jalur pantai Asia dari Asia Barat, dan pada sekitar 60.000 sampai 70.000 tahun yang lalu telah mencapai Pulau Papua dan Australia.[3] Mereka, yang berfenotipe kulit gelap dan rambut ikal rapat, menjadi nenek moyang penduduk asli Melanesia (termasuk Papua) sekarang dan membawa kultur kapak lonjong (Paleolitikum). Gelombang pendatang berbahasa Austronesia dengan kultur Neolitikum datang secara bergelombang sejak 3000 SM dari Cina Selatan melalui Formosa dan Filipina membawa kultur beliung persegi (kebudayaan Dongson). Proses migrasi ini merupakan bagian dari pendudukan Pasifik. Kedatangan gelombang penduduk berciri Mongoloid ini cenderung ke arah barat, mendesak penduduk awal ke arah timur atau berkawin campur dengan penduduk setempat dan menjadi ciri fisik penduduk Maluku serta Nusa Tenggara. Pendatang ini membawa serta teknik-teknik pertanian, termasuk bercocok tanam padi di sawah (bukti paling lambat sejak abad ke-8 SM), beternak kerbau, pengolahan perunggu dan besi, teknik tenun ikat, praktik-praktik megalitikum, serta pemujaan roh-roh (animisme) serta benda-benda keramat (dinamisme). Pada abad pertama SM sudah terbentuk permukiman-permukiman serta kerajaan-kerajaan kecil, dan sangat mungkin sudah masuk pengaruh kepercayaan dari India akibat hubungan perniagaan.")

In [4]:
print(translated)

Please summarize the following sentences: The first Homo sapiens were thought to have entered the archipelago 100,000 years ago via the Asian coastal route from West Asia, and around 60,000 to 70,000 years ago had reached New Guinea and Australia.[3] They, who had the phenotype of dark skin and tightly curled hair, became the ancestors of the indigenous people of Melanesia (including Papua) today and brought the culture of oval axes (Paleolithic). Waves of Austronesian-speaking immigrants with Neolithic culture came in waves since 3000 BC from South China via Formosa and the Philippines bringing the square pickaxe culture (Dongson culture). This migration process is part of the Pacific occupation. The arrival of this wave of people with Mongoloid characteristics tended towards the west, pushing the initial population towards the east or intermarrying with the local population and becoming a physical characteristic of the people of Maluku and Nusa Tenggara. These immigrants brought with

In [5]:
output_2 = model.generate(translated)
print(output_2)

In [6]:
translated_2 = GoogleTranslator(source='en', target='id').translate(output_2)

In [7]:
print(translated_2)

In [8]:
class util:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")
        
    def batch(self, iterable, n=1):
        l = len(iterable)
        for ndx in range(0, l, n):
            yield iterable[ndx:min(ndx + n, l)]

    def cos_sim(x,y):
      return sentence_transformers.util.cos_sim(x,y)

In [9]:
class Reducer:
    def __init__(self, model_name='msmarco-MiniLM-L-6-v3', device="cpu"):
        self.device = device if torch.cuda.is_available() else "cpu"
        self.embmodel = SentenceTransformer(model_name).to(self.device)
        self.util = util()
        if self.device != "cpu":
          spacy.require_gpu()

    def reduce_small(self, text, target_embedding, num_sents=5):
        sentences = [sent.text for sent in self.util.nlp(text).sents]
        new_texts = []
        for sent in sentences:
            new_text = [s for s in sentences]
            new_text.remove(sent)
            new_texts.append(new_text)
        new_texts = [" ".join(new_text) for new_text in new_texts]
        new_text_embeddings = self.embmodel.encode(new_texts, convert_to_tensor=True, show_progress_bar=False)
        cos_scores = util.cos_sim(target_embedding, new_text_embeddings)[0]
        top_indices = torch.topk(cos_scores, k=len(new_texts))[1]
        top_texts = [new_texts[i] for i in top_indices]
 
        top_text = top_texts[0]
        top_sentences = [sent.text for sent in self.util.nlp(top_text).sents]
        if len(top_sentences)<=num_sents:
            return top_text
        else:
            return self.reduce_small(top_text, target_embedding, num_sents)

    def reduce_large(self, big_text, target_embedding, num_sents=5):
        big_sentences = [sent.text for sent in self.util.nlp(big_text).sents]
        parts = []
        for part in self.util.batch(big_sentences, 10):
            parts.append(self.reduce_small(' '.join(part), target_embedding, num_sents))
        new = ' '.join(parts)
        if len([sent.text for sent in self.util.nlp(new).sents])<=num_sents:
            return new
        elif len(parts)==1:
            return self.reduce_small(new, target_embedding, num_sents)
        else:
            return self.reduce_large(new, target_embedding, num_sents)

    def reduce(self, huge_text, target, num_sents=5):
        target_embedding = self.embmodel.encode(target, convert_to_tensor=True, show_progress_bar=False)
        #huge_sentences = [sent.text for sent in self.self.util.nlp(huge_text).sents]
        huge_sentences = [x+"." for x in huge_text.split(". ")]
        parts = []
        for part in  self.util.batch(huge_sentences, 4):
            parts.append(" ".join(part))
        new_text_embeddings = self.embmodel.encode(parts, convert_to_tensor=True, show_progress_bar=False)
        cos_scores = util.cos_sim(target_embedding, new_text_embeddings)[0]
        top_indices = torch.topk(cos_scores, k=len(parts))[1]
        top_texts = [parts[i] for i in top_indices][:3]
        
        chronological = []
        for part in parts:
            if part in top_texts:
                chronological.append(part)
        reduction = self.reduce_large(" ".join(chronological), target_embedding, num_sents)
        return reduction

    def summarize(self, text, num_sents=5):
        return self.reduce(text, text, num_sents=num_sents)

    def reduce_pdf(self, pdf_path, target, num_sents=5):
        with open(pdf_path, 'rb') as f:
            pdf = PyPDF2.PdfFileReader(f)
            text = ""
            for page in pdf.pages:
                text += page.extractText()
        reduction = self.reduce(text, target, num_sents)
        return reduction

In [10]:
class QAReducer:
    def __init__(self, model_name='cross-encoder/qnli-distilroberta-base', device="cpu"):
        self.device = device if torch.cuda.is_available() else "cpu"
        self.conf_model = CrossEncoder('cross-encoder/qnli-distilroberta-base',device=0)
        self.util = util()
        if self.device != "cpu":
          spacy.require_gpu()

    def reduce_small(self, text, query, num_sents=5):
        sentences = [sent.text for sent in self.util.nlp(text).sents]
        new_texts = []
        for sent in sentences:
            new_text = [s for s in sentences]
            new_text.remove(sent)
            new_texts.append(new_text)
        new_texts = [" ".join(new_text) for new_text in new_texts]
        cos_scores = self.conf_model.predict([(query, new_text) for new_text in new_texts])
        sorted = torch.topk(torch.tensor(cos_scores), k=len(new_texts))
        top_indices = sorted[1]
       
        conf = sorted[0][0]
        top_texts = [new_texts[i] for i in top_indices]
      
        top_text = top_texts[0]
        top_sentences = [sent.text for sent in self.util.nlp(top_text).sents]
        if len(top_sentences)<=num_sents:
            return top_text, conf
        else:
            return self.reduce_small(top_text, query, num_sents)

    def reduce_large(self, big_text, query, num_sents=5):
        big_sentences = [sent.text for sent in self.util.nlp(big_text).sents]
        parts = []
        conf = 0
        for part in self.util.batch(big_sentences, 10):
            out, conf = self.reduce_small(' '.join(part), query, num_sents)
            parts.append(out)
        new = ' '.join(parts)
        if len([sent.text for sent in self.util.nlp(new).sents])<=num_sents:
            return new, conf
        elif len(parts)==1:
            return self.reduce_small(new, query, num_sents)
        else:
            return self.reduce_large(new, query, num_sents)

    def reduce(self, huge_text, query, num_sents=5):
        huge_sentences = [x+"." for x in huge_text.split(". ")]
        parts = []
        for part in self.util.batch(huge_sentences, 4):
            parts.append(" ".join(part))
        cos_scores = self.conf_model.predict([(query, part) for part in parts])
        top_indices = torch.topk(torch.tensor(cos_scores), k=len(parts))[1]
        top_texts = [parts[i] for i in top_indices][:3]
        chronological = []
        for part in parts:
            if part in top_texts:
                chronological.append(part)
        reduction, conf = self.reduce_large(" ".join(chronological), query, num_sents)
        return reduction, conf

In [11]:
def remove_special_characters(text):
    characters_to_remove = ['*', '\n', '===', '==', '=', '#', '$']
    for char in characters_to_remove:
        text = text.replace(char, '')
    return text

In [12]:
def remove_xa0_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    for tag in soup.find_all('p'):
        if tag.string == '\xa0':
            tag.decompose()  # Menghapus tag dari teks
    return str(soup)

In [13]:
def remove_h2_with_text(text, target_text):
    soup = BeautifulSoup(text, 'html.parser')
    for tag in soup.find_all('h2'):
        if target_text in tag.get_text():
            tag.decompose()  # Menghapus tag dari teks
    return str(soup)

In [14]:
# import the sentence transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [15]:
def render_table_to_image(table_html, output_path):
    options = {
        'format': 'png',
        'encoding': 'utf-8',
    }

    imgkit.from_string(table_html, output_path, options=options)

In [16]:
new_data_test = []
data_test =[
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#h-07042022-analisis-target-akuisisi-dan-aktivasi-merchant-qris\"></a> [07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS</h1>",
        "text_content": "<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<th>Direktorat</th>\n<td> Digital Insight</td>\n</tr>\n<tr>\n<th>Sub Direktorat</th>\n<td>MIS 1 </td>\n</tr>\n<tr>\n<th>Analysis Date</th>\n<td>7 April 2022</td>\n</tr>\n<tr>\n<th>Analysis status</th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>Document status </th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>PIC  Analyst</th>\n<td> Ghifari Daulagiri</td>\n</tr>\n<tr>\n<th>Stakeholders</th>\n<td>DIgital Banking</td>\n</tr>\n<tr>\n<th>Reference Document</th>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#version-history\"></a> Version History</h2>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><i><strong>Version</strong></i></td>\n<td><i><strong>Date</strong></i></td>\n<td><i><strong>Description</strong></i></td>\n</tr>\n<tr>\n<td> 1.0</td>\n<td>2022-04-07 </td>\n<td>initial document</td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h1><a href=\"#h-07042022-analisis-target-akuisisi-dan-aktivasi-merchant-qris\"></a> [07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS</h1>\n<h2><a href=\"#latar-belakang-dan-tujuan\"></a> Latar Belakang dan Tujuan</h2>\n<p>Analisis ini dibuat sebagai dasar pembuatan dan desain kontrol untuk strategi akuisisi dan aktivasi merchant QRIS.</p>\n<p>Target full year 2022 adalah akuisisi 100.000 merchant QRIS, dan 1 Trilyun CASA. Dimana transaksi QRIS adalah salah satu channel untuk mencapai target CASA. Oleh karena itu perlu dilakukan analisis untuk menyusun strategi akuisisi dan aktivasi merchant QRIS, menurunkan teknis pelaksanaan di lapangan, dan desain &amp; mekanisme kontrol pelaksanaannya.</p>\n<h2><a href=\"#temuan-utama\"></a> Temuan Utama</h2>\n<h3><a href=\"#simulasi-target\"></a> Simulasi Target</h3>\n<p>Berangkat dari target 100 ribu customer dan asumsi 1T target CASA bisa didapatkan <strong>seluruhnya</strong> dari transaksi QRIS. Skema simulasi menggunakan skema Referral yang sudah terdefinisi di dokumen ‘Target 1 Juta Nasabah’.</p>\n<h3><a href=\"#simulasi-kondisi-ideal\"></a> Simulasi kondisi ideal</h3>\n<p>Dalam simulasi ini, metric yang harus dijaga untuk dapat mencapai tujuan adalah:</p>\n<p>Metric akuisisi merchant:</p>\n<ol>\n<li>Rasio Merchant Aktif: 20%</li>\n<li>Merchant MGM % Merchant Aktif: 30%</li>\n<li>Rata-rata Akuisisi per Merchant MGM: 5 merchant</li>\n</ol>\n<p>Metric aktivasi merchant:</p>\n<ol>\n<li>Banyaknya Transaksi per Merchant: 150 transaksi</li>\n<li>Rata-rata Nilai Transaksi: IDR 100.000</li>\n</ol>\n<figure><img src=\"/aktivasimerchantqris/untitled.png\"/></figure>\n<p>Hasil simulasi menunjukkan performa hingga 130% untuk akuisisi merchant dan 120% untuk transaksi.</p>\n<h3><a href=\"#simulasi-kondisi-gradual\"></a> Simulasi kondisi gradual</h3>\n<p>Dalam simulasi ini, terdapat asumsi dimana peningkatan secara gradual jumlah merchant MGM yaitu dari 5% di April, meningkat hingga 30% di September. Asumsi metric yang lain adalah sama.</p>\n<figure><img src=\"/aktivasimerchantqris/untitled_1.png\"/></figure>\n<p>Hasil simulai menunjukkan performa hanya 77% dan 73% untuk akuisisi dan transaksi, dimana keduanya masih jauh dari target.</p>\n<h3><a href=\"#simulai-kondisi-realistis\"></a> Simulai kondisi realistis</h3>\n<p>Dalam simulasi ini, ditambahkan asumsi pertumbuhan merchant aktif secara gradual mulai dari 13% (data per Maret) hingga 25% di bulan Juli.</p>\n<figure><img src=\"/aktivasimerchantqris/untitled_2.png\"/></figure>\n<p>Hasil simulasi menunjukkan performa di 100% dan 98% untuk akuisisi dan transaksi.</p>\n<h3><a href=\"#kondisi-aktual-posisi-maret-2022\"></a> Kondisi Aktual (Posisi Maret 2022)</h3>\n<p>Secara holistik, Rasio Merchant Aktif adalah 12.8%, Banyaknya Transaksi per Merchant adalah 30, dan Rata-rata Nilai Transaksi adalah IDR 90.886, dimana ketiganya perlu diperbaiki untuk memenuhi kriteria simulasi.</p>\n<p>Mengenai kondisi metric cabang:</p>\n<p>Rasio Merchant Aktif:</p>\n<figure><img src=\"/aktivasimerchantqris/untitled_3.png\"/></figure>\n<p>Transaksi per Merchant</p>\n<figure><img src=\"/aktivasimerchantqris/untitled_4.png\"/></figure>\n<p>Rata-rata Nilai Transaksi:</p>\n<figure><img src=\"/aktivasimerchantqris/untitled_5.png\"/></figure>\n<p>Metric yang paling utama untuk diperbaiki adalah metric aktivasi, yaitu Rasio Merchant Aktif dan Transaksi per Merchant.</p>\n<p> </p>\n<h2><a href=\"#rekomendasi\"></a> Rekomendasi</h2>\n<p>Kunci keberhasilan simulasi adalah peran Merchant yang aktif sebagai perpanjangan tangan Cabang; baik untuk mengakuisisi merchant lain, dan mempromosikan penggunaan QRIS BAG. Untuk itu perlu dipastikan hubungan baik Cabang dan Merchant sehingga terjalin kerjasama yang berdampak positif terhadap bisnis Merchant dan Bank.</p>\n<p>Untuk menunjang kunci keberhasilan tersebut, perlu adanya tracking dan definisi target untuk metric tertentu kepada cabang, baik itu dalam aspek akuisisi maupun aktivasi.</p>\n<p>Perlu adanya gamifikasi sebagai sarana berkompetisi antar cabang, dan perlu diadakan reward bagi cabang yang membuktikan prestasinya.</p>\n<p>Oleh karena itu, direkomendasikan model pemantauan metric berupa kuadran capaian dengan poros-poros:</p>\n<ol>\n<li>Metric Akuisisi: Total Merchant, dimana pencapaian cabang dibagi berpusat pada median akuisisi cabang.</li>\n<li>Metric Aktivasi: Raiso Merchant aktif, dimana pencapaian cabang dibagi berpusat pada target user aktif (20%)</li>\n</ol>\n<figure><img src=\"/aktivasimerchantqris/untitled_6.png\"/></figure>\n<p>Target user aktif harus rutin dikaji berdasarkan realisasi capaian cabang dan kebutuhan pemenuhan target tahun 2022</p>\n<p>Perlu adanya arahan rutin ke cabang sebagai sarana cabang untuk mengidentifikasi merchant yang perlu distimulasi untuk kembali aktif dan/atau diberikan apresiasi karena telah menjadi merchant yang loyal kepada BAG.</p>\n",
        "tanggal_content": "2022-12-28T02:53:36.352Z",
        "author_content": "Rahmi",
        "direktorat_content": "Digital Insight",
        "PIC_Analyst": "Ghifari Daulagiri",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/AnalisisTargetAkuisisidanAktivasiMerchantQRIS"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#h-08042022-database-promo-ideation\"></a> [08042022] Database Promo Ideation</h1>",
        "text_content": "<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<th>Direktorat</th>\n<td> Digital Insight</td>\n</tr>\n<tr>\n<th>Sub Direktorat</th>\n<td>MIS 1 </td>\n</tr>\n<tr>\n<th>Analysis Date</th>\n<td>8 April 2022</td>\n</tr>\n<tr>\n<th>Analysis status</th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>Document status </th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>PIC  Analyst</th>\n<td> Ghifari Daulagiri</td>\n</tr>\n<tr>\n<th>Stakeholders</th>\n<td>DIgital Banking</td>\n</tr>\n<tr>\n<th>Reference Document</th>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#version-history\"></a> Version History</h2>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><i><strong>Version</strong></i></td>\n<td><i><strong>Date</strong></i></td>\n<td><i><strong>Description</strong></i></td>\n</tr>\n<tr>\n<td> 1.0</td>\n<td>2022-04-07 </td>\n<td>initial document</td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h1><a href=\"#h-08042022-database-promo-ideation\"></a> [08042022] Database Promo Ideation</h1>\n<p>Berdasarkan <a href=\"http://10.0.231.117:3000/en/Insight/AnalisisTargetAkuisisidanAktivasiMerchantQRIS\">[07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS</a> , rekomendasi promo yang ditargetkan adalah</p>\n<ol>\n<li>Metric Akuisisi: Total Merchant, dimana pencapaian cabang dibagi berpusat pada median akuisisi cabang.</li>\n<li>Metric Aktivasi: Raiso Merchant aktif, dimana pencapaian cabang dibagi berpusat pada target user aktif (20%)</li>\n</ol>\n<p>Dengan skema pemberian promo berdasarkan mapping kuadran ini</p>\n<figure><img src=\"/database_promo.png\"/></figure>\n<p> </p>\n<figure>\n<table>\n<thead>\n<tr>\n<th>No</th>\n<th>Promo</th>\n<th>Bentuk Promo</th>\n<th>Mekanisme Promo</th>\n<th>Achievement</th>\n</tr>\n</thead>\n<tbody>\n<tr>\n<td>0</td>\n<td>promo apa yang diusulkan?</td>\n<td>bentuknya diskon, cashback, atau apa?</td>\n<td>bagiamana skema promo-nya?</td>\n<td>apa yang mission yang diachieve untuk kita</td>\n</tr>\n<tr>\n<td>1</td>\n<td>Merchant discount</td>\n<td>Discount or cashback</td>\n<td>User mendapatkan sejumlah X% discount max A jika belanja di merchant tertentu (yang sudah ditentukan merchantnya) dengan AGI mobile/agicash</td>\n<td>Meningkatkan activation dan retention user; target Activation Zone dan Hall of Fame</td>\n</tr>\n<tr>\n<td>2</td>\n<td>Mission Referral</td>\n<td>Cashback &amp; Gamification</td>\n<td>Referrer akan mendapatkan cashback sejumlah X jika TELAH mencapai misi refer P orang untuk memuat akun agi mobile/agicash</td>\n<td>Meningkatkan acquisition; Dengan gamification user akan distimulasi untuk melakukan misi mencapai target cashback; target Acquisition Zone</td>\n</tr>\n<tr>\n<td>3</td>\n<td>Mission Referral Transaction</td>\n<td>Cashback &amp; Gamification</td>\n<td>Referrer dan Referee mendapatkan cashback sejumlah X jika Referee telah membuat transaksi pertama</td>\n<td>Meningkatkan acquisition dan activation; Dengan gamification user akan distimulasi untuk melakukan misi mencapai target cashback; target Acquisition Zone dan Activation Zone</td>\n</tr>\n<tr>\n<td>4</td>\n<td>Mission Transaction</td>\n<td>Cashback &amp; Gamification</td>\n<td>User mendapatkan cashback sejumlah X jika telah mencapai misi membuat transaksi sebanyak N kali</td>\n<td>Meningkatkan activation; Dengan gamification user akan distimulasi untuk melakukan misi mencapai target cashback; target Activation Zone</td>\n</tr>\n<tr>\n<td>5</td>\n<td>Mission Create AGI Account</td>\n<td>Cashback &amp; Gamification</td>\n<td>User mendapatkan cashback sejumlah X jika mendaftarkan akun menjadi nasabah AGI</td>\n<td>Meningkatkan activation AGI acccount (Tabungan Icon bukan ya???)</td>\n</tr>\n<tr>\n<td>6</td>\n<td>Discount admin fee connect to emoney</td>\n<td>Discount</td>\n<td>User mendapatkan discount admin fee untuk topup emoney; misalnya GOPAY, OVO (*lumayan 1000)</td>\n<td>Meningkatkan activation dan retention user; target Activation Zone dan Hall of Fame</td>\n</tr>\n<tr>\n<td>7</td>\n<td>Voucher redeem</td>\n<td>Cashback</td>\n<td>AGI menyediakan sejulah voucher untuk di redeem, user bisa kapan saja redeem; voucher juga memiliki waktu expired</td>\n<td>Meningkatkan activation dan retention user; target Activation Zone dan Hall of Fame</td>\n</tr>\n<tr>\n<td>8</td>\n<td>Donasi</td>\n<td>-</td>\n<td>Menambahkan fitur donasi ketika melakukan pembayaran</td>\n<td>Meningkatkan value “for others”</td>\n</tr>\n<tr>\n<td>9</td>\n<td>Program mendapatkan ide dari internal, sebulan sekali atau 2 bulan sekali</td>\n<td>Voucher atau cashback di Agi mobile</td>\n<td>Membuat program untuk intenal AGI (ditentukan siapa); siapa pun yang punya ide untuk meningkatkan acquisition atau activation atau program yang bsa membantu mencapai target, maka akan mendaatkan voucher yang bisa di redeem</td>\n<td>Meningkatkan bounding di internal AGI dan gotong royong mencapat target; sekaligus mengajak karyawan menggunakan agi mobile dan bertransaksi dengan agi mobile</td>\n</tr>\n<tr>\n<td>10</td>\n<td>Hackethon Bug Bountie atau Deasign</td>\n<td>Voucher atau cashback di Agi mobile</td>\n<td>Membuat program bug bountie atau design aplikasi untuk internal karyawan</td>\n<td>Meningkatkan bounding di internal AGI dan gotong royong mencapat target AGI; sekaligus mengajak karyawan menggunakan agi mobile dan bertransaksi dengan agi mobile</td>\n</tr>\n<tr>\n<td>11</td>\n<td>Posting terkait AGI mobile dan undian pemenang</td>\n<td>Voucher atau cashback di Agi mobile</td>\n<td>Membuat program posting sesuatu yang kreatif atau yang sudah ditentukan (misal ada event tertentu); lalu post di instagram</td>\n<td>Meningkatkan bounding di internal AGI dan gotong royong mencapat target AGI; sekaligus mengajak karyawan menggunakan agi mobile dan bertransaksi dengan agi mobile</td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n",
        "tanggal_content": "2022-12-28T08:26:29.398Z",
        "author_content": "Rahmi",
        "direktorat_content": "Digital Insight",
        "PIC_Analyst": "Ghifari Daulagiri",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/DatabasePromoIdeation"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#h-10032022-analisis-transaksi-untuk-targeted-marketing\"></a> [10032022] Analisis Transaksi Untuk Targeted Marketing</h1>",
        "text_content": "<h1><a href=\"#h-10032022-analisis-transaksi-untuk-targeted-marketing\"></a> [10032022] Analisis Transaksi Untuk Targeted Marketing</h1>\n<h2><a href=\"#latar-belakang-dan-tujuan\"></a> Latar Belakang dan Tujuan</h2>\n<p>Saat ini manajemen <i>Lead Base</i> kita belum optimal untuk membuat sebuah keputusan bisnis sehingga Ketika melakukan aktivitas sales ke nasabah/calon nasabah, eksekusi di lapangan masih sporadis dan tidak terfokuskan ke target pasar yang dituju</p>\n<p>Analisis ini dibuat untuk memberikan insight dalam penentuan fokus dan opsi aktivitas pre-sales dan sales ke nasabah/calon nasabah.</p>\n<h2><a href=\"#temuan-utama\"></a> Temuan Utama</h2>\n<p>Data yang digunakan adalah data nasabah snapshot per Maret 2022, data finansial sepanjang Februari 2022, data transaksi sepanjang bulan Januari 2022.</p>\n<h2><a href=\"#distribusi-usia-nasabah-dan-perilaku-bertransaksi\"></a> Distribusi Usia Nasabah dan Perilaku Bertransaksi</h2>\n<figure><img src=\"/transaksiuntuktargetedmarketing/untitled.png\"/></figure>\n<p>Distribusi nasabah terkonsentrasi di usia produktif 25-40 tahun. Ini sejalan dengan kondisi dimana rekening artha graha banyak terdiri dari rekening payroll.</p>\n<p>Dana tersebut juga merupakan dana aktif, dalam artian perputaran dana masuk dan keluar (e.g. transaksi, transfer, pembayaran) terjadi dan lancar.</p>\n<p>Statemen tersebut juga dapat divalidasi dengan konsentrasi dana simpanan, dimana konsentrasi AUM justru tersebar di distribusi usia yang lebih tua.</p>\n<figure><img src=\"/transaksiuntuktargetedmarketing/untitled_1.png\"/></figure>\n<p>Artinya analisis lebih lanjut harus dilakukan di segmen usia 25-40 tahun apabila tujuan analisis adalah menemukan pola transaksi dan mendorong perilaku bertransaksi menggunakan solusi pembayaran AGI Mobile.</p>\n<h2><a href=\"#korelasi-usia-jenis-pekerjaan-dan-transaksi\"></a> Korelasi Usia, Jenis Pekerjaan, dan Transaksi</h2>\n<figure><img src=\"/transaksiuntuktargetedmarketing/untitled_2.png\"/></figure>\n<p>Diatas adalah visualisasi 10 jenis pekerjaan dengan nasabah terbanyak.</p>\n<blockquote>\n<p>💡 Terlihat bahwa konsentrasi nasabah ada di Administrasi Umum, Buruh, Pengamanan, Pegawai Swasta, dan Hospitality.</p>\n</blockquote>\n<figure><img src=\"/transaksiuntuktargetedmarketing/untitled_3.png\"/></figure>\n<p>Selanjutnya kita akan kembali menguatkan fokus ke segmen tersebut. </p>\n<blockquote>\n<p>💡 Mayoritas perilaku transaksi adalah Transfer, baik pemindahbukuan atau ke bank lain. Ini adalah hal yang <strong>tidak diinginkan</strong>.</p>\n</blockquote>\n<p>Apabila mayoritas nasabah hanya melakukan transfer dana keluar BAG, hal yang perlu diketahui lagi adalah segmen nasabah mana saja yang saat ini menggunakan produk BAG untuk melakukan transaksi pembayaran atau top-up.</p>\n<figure><img src=\"/transaksiuntuktargetedmarketing/untitled_4.png\"/></figure>\n<p>Terlihat bahwa nasabah dengan jenis pekerjaan Administrasi Umum, Buruh, Pengamanan, Pegawai Swasta, dan Hospitality termasuk segmen nasabah yang memiliki perilaku bertransaksi pembayaran dan Top-Up e-wallet.</p>\n<p>Selain 5 segmen diatas, pelajar dan mahasiswa juga memiliki appetite yang tinggi untuk melakukan pembayaran.</p>\n<figure><img src=\"/transaksiuntuktargetedmarketing/untitled_5.png\"/></figure>\n<h2><a href=\"#kesimpulan\"></a> Kesimpulan</h2>\n<p>Ada beberapa hal yang perlu digarisbawahi:</p>\n<ol>\n<li>Terdapat appetite untuk menggunakan platform BAG dalam melakukan pembayaran</li>\n<li>Segmen nasabah yang potensial untuk disasar adalah:<ol>\n<li>Pekerja white-collar dengan income menengah kebawah (&lt;10 juta)</li>\n<li>Pekerja blue-collar</li>\n<li>Buruh</li>\n<li>Pengamanan</li>\n</ol>\n</li>\n<li>Segmen pelajar/mahasiswa dengan usia dibawah produktif (15-25 tahun) memiliki appetite yang tinggi dalam melakukan transaksi pembayaran; walau dana yang masuk ke rekening relatif kecil daripada segmen usia produktif.</li>\n</ol>\n<h2><a href=\"#rekomendasi\"></a> Rekomendasi</h2>\n<p>Untuk rekomendasi merchant, selain gerai makanan/kuliner, fokus marketing dan akuisisi merchant harus mendukung dan dirasakan menarik oleh segmen nasabah diatas. Contoh:</p>\n<ol>\n<li>Warmindo di sekitar lokasi pabrik/kost</li>\n<li>Bengkel modifikasi sepeda motor</li>\n<li>Distributor sparepart dan aksesoris sepeda motor</li>\n<li>Distribusi sembako</li>\n</ol>\n<p>Perlu riset dan survey lebih lanjut ke segmen nasabah tersebut untuk mencari tahu lebih jauh mengenai preferensi transaksi mereka.</p>\n<p>Perlu riset dan survey lebih lanjut terhadap kompetitor: OVO dan LINKAJA. Terutama sub-produk apa dari kompetitor yang dipakai oleh nasabah BAG.</p>\n<h2><a href=\"#action-items\"></a> Action Items</h2>\n<p>[ ] <strong>[PIC: Agnes &amp; Dian]</strong> Penyusunan program acquisition &amp; promo untuk:</p>\n<ol>\n<li>Pulsa &amp; paket data, Games —&gt; untuk segment pelajar &amp; mahasiswa —&gt; Mobile Legend, Free Fire, Apex Legend</li>\n<li>Bengkel Modifikasi &amp; Aksesori Motor —&gt; untuk segment buruh dan pekerja</li>\n<li>Sembako —&gt; untuk karyawan group dan all existing nasabah</li>\n<li>Warmindo (warung indomie) —&gt; untuk segment buruh dan pekerja</li>\n<li>% Agi Mobile User</li>\n<li>New Customer/Conversion</li>\n<li>Feebased income</li>\n<li>Cost/Burn rate</li>\n</ol>\n<p>[x] [PIC: Bu Erika] Penyusunan naskah survey untuk riset lebih lanjut tentang use-case mobile banking/payment di segmen tersebut; Target: Senin 14 Maret 2022</p>\n<p>[ ] [PIC: Bu Erika + Indri] Eksekusi survey untuk existing nasabah atau pengguna didalam group AG</p>\n<p>Target: 361 data points</p>\n<p>[ ] [PIC: Agnes] Eksekusi survey untuk target segment diluar group</p>\n<p>Target: 361 data points</p>\n",
        "tanggal_content": "2022-11-21T07:47:10.918Z",
        "author_content": "Administrator",
        "direktorat_content": "Tidak ada direktorat",
        "PIC_Analyst": "Tidak ada PIC Analyst",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/analisistransaksiuntuktargetedmarketing"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#analisis-merchant-qris-demografi\"></a> Analisis Merchant QRIS Demografi</h1>",
        "text_content": "<h1><a href=\"#analisis-merchant-qris-demografi\"></a> Analisis Merchant QRIS Demografi</h1>\n<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<th>Direktorat</th>\n<td> Digital Insight</td>\n</tr>\n<tr>\n<th>Sub Direktorat</th>\n<td>MIS 1 </td>\n</tr>\n<tr>\n<th>Analysis Date</th>\n<td>7 Juli 2022</td>\n</tr>\n<tr>\n<th>Analysis status</th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>Document status </th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>PIC  Analyst</th>\n<td> Rahmi Julianasari</td>\n</tr>\n<tr>\n<th>Stakeholders</th>\n<td>DIgital Banking</td>\n</tr>\n<tr>\n<th>Reference Document</th>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#version-history\"></a> Version History</h2>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><i><strong>Version</strong></i></td>\n<td><i><strong>Date</strong></i></td>\n<td><i><strong>Description</strong></i></td>\n</tr>\n<tr>\n<td> 1.0</td>\n<td>2022-04-07 </td>\n<td>initial document</td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p>Source:</p>\n<p><a href=\"/data insight/merchant_bahan_meeting_202304_(1).xlsx\">/data insight/merchant_bahan_meeting_202304_(1).xlsx</a></p>\n<h3><a href=\"#grouping-category-cleansing\"></a> Grouping Category (Cleansing)</h3>\n<pre v-pre=\"true\"><code>                  , CASE\n                        WHEN REGEXP_LIKE(MQ.MERCHANT_NAME, 'warteg|KUE|ayam|bakso|warteg|soto|makan|burger|kfc|food|sapi|bebek|mcd|nasi|cafe|sate|rawon|gudeg|pempek|rendang|gado|martabak|udang|mie|ikan|jamur|otak|goreng|bubur|rujak|pecel|laksa|kerupuk|lapis|puding|gulai|pepes|perkedel|sayur|sop|lontong|karedok|ketupat|tahu|gorengan|serabi|kolak|lemang|batagor|telor|semur|coffee|kopi|angkringan|dimsum|kedai|butter|catering|shihlin|solaria|fresh|xi bo ba|jco|es|janji jiwa|steak|starbucks|bakmie|shigeru|kintan|gelato|minang|crepes|onigiri|geprek|teh|kokumi|warteq|sushi|mkn|shushu|buah|kkuldak|chicken|corndog|dapur|bakery|buah|truffle|kemchicks|kantin|nyapii|kebab|makaroni|tongseng|susu|ikkudo|coffe|bakmi|koi|crepes|wingstop|ngopi|chatime|reddog|pagi sore|tea|corner|rice|koppi|warkop|ramen|eat|waroeng|masakan|wearung|jus|delico|stroberi|shabu|pig|kwetiau|kitchen|baso|lumpia|warunk|takoyaki|pizza|drinks|dessert|rotbak|auntie anne|babi|gokana|crispy|montato|yoshinoya|boba|cendol|dapoer|bbq|menantea|A&amp;W|bobain|sushi|bak|roti|chashio|uncle k|alpukat|dum|cilok|banana|wendy|kokoho|magal|kantin|juice|gokana|warunk|cream|hakata|steamboat|wing|puyo|wasabi|waroeng|potato|nasgorgil|nasgor|snack|ikkudo|saoenk|doner|broadway|jfc|imperial|kitamura|subway|breadtalk|kangtebu|shabu|talas|soerabi|eatlah|sandwich|xing fu tang|crispy|the harvest|warkop|zenbu|toast|roti|aroma|bkso|foodhall|warsun|pappajack\n|rm') THEN 'FOOD &amp; BEVERAGE'\n                        WHEN REGEXP_LIKE(MQ.MERCHANT_NAME,\n                                         'warung|mart|circle k|alfamart|alfamidi|market|kios|miniso|lawson|daiso|indomaret')\n                            THEN 'VARIETY_STORE'\n                        WHEN REGEXP_LIKE(MQ.MERCHANT_NAME,\n                                         'pay|tokko|bayar|tokopedia|indodax|blibli|itemku|guardian')\n                            THEN 'E-COMMERCE'\n                        WHEN MQC.DESCRIPTION = 'Regular' THEN 'REGULAR'\n                        WHEN MQC.DESCRIPTION = 'FAST FOOD RESTAURANTS' THEN 'FOOD &amp; BEVERAGES'\n                        WHEN MQC.DESCRIPTION = 'MISCELLANEOUS FOOD SHOPS-CONVENIENCE AND SPECIALITY RETAIL OUTLETS'\n                            THEN 'RETAIL OUTLETS'\n                        WHEN MQC.DESCRIPTION = 'SERVICE STATION' THEN 'BUILDING, ELECTRONICS, AUTOMOTIVE SERVICES'\n                        WHEN MQC.DESCRIPTION = 'GROCERIES AND SUPERMARKETS' THEN 'GROCERIES AND SUPERMARKETS'\n                        WHEN MQC.DESCRIPTION = 'OTHERS-NOT ELSEWHERE CLASSIFIED' THEN 'OTHERS-NOT ELSEWHERE CLASSIFIED'\n                        WHEN MQC.DESCRIPTION = 'MISCELLANEOUS AND SPECIALITY RETAIL OUTLETS' THEN 'RETAIL OUTLETS'\n                        WHEN MQC.DESCRIPTION = 'FAMILY CLOTHING SHOPS' THEN 'CLOTHING FASHION'\n                        WHEN MQC.DESCRIPTION = 'ACCOUNTING, AUDITING AND BOOKKEEPING SERVICES' THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'BEAUTY AND BARBER SHOPS' THEN 'BEAUTY AND BARBER SHOPS'\n                        WHEN MQC.DESCRIPTION = 'VARIETY STORES' THEN 'VARIETY STORES'\n                        WHEN MQC.DESCRIPTION = 'MEN’S AND WOMEN’S CLOTHING SHOPS' THEN 'CLOTHING FASHION'\n                        WHEN MQC.DESCRIPTION = 'WOMEN’S ACCESSORY AND SPECIALITY SHOPS' THEN 'BEAUTY SHOP'\n                        WHEN MQC.DESCRIPTION = 'BAKERIES' THEN 'FOOD &amp; BEVERAGES'\n                        WHEN MQC.DESCRIPTION = 'ELECTRONICS SHOPS' THEN 'HOME SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'CLOTHING RENTALS - COSTUMES, UNIFORMS AND FORMAL WEAR'\n                            THEN 'CLOTHING FASHION'\n                        WHEN MQC.DESCRIPTION = 'AIRLINES AND AIR CARRIERS' THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'WOMEN’S READY-TO-WEAR SHOPS' THEN 'CLOTHING FASHION'\n                        WHEN MQC.DESCRIPTION =\n                             'DRINKING PLACES (ALCOHOLIC BEVERAGES) - BARS, TAVERNS, NIGHT-CLUBS, COCKTAIL LOUNGES AND DISCOTHÈQUES'\n                            THEN 'FOOD &amp; BEVERAGES'\n                        WHEN MQC.DESCRIPTION = 'TELECOMMUNICATION EQUIPMENT AND TELEPHONE SALES' THEN 'TELCO &amp; PEYMENT'\n                        WHEN MQC.DESCRIPTION = 'FOOD &amp; BEVERAGE' THEN 'FOOD &amp; BEVERAGE'\n                        WHEN MQC.DESCRIPTION = 'COSMETIC SHOPS' THEN 'BEAUTY SHOP'\n                        WHEN MQC.DESCRIPTION = 'TRANSPORTATION SERVICES (NOT ELSEWHERE CLASSIFIED)'\n                            THEN 'BUILDING, ELECTRONICS, AUTOMOTIVE SERVICES'\n                        WHEN MQC.DESCRIPTION = 'GOVERNMENT SERVICES (NOT ELSEWHERE CLASSIFIED)' THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'GOVERMENTS SERVICES (NOT ELSEWHERE CLASSIFIED)' THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'Government Service' THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'TAILORS, SEAMSTRESSES, MENDING AND ALTERATIONS' THEN 'CLOTHING FASHION'\n                        WHEN MQC.DESCRIPTION = 'AUTOMOTIVE SERVICE SHOPS (NON-DEALER)' THEN 'AUTOMOTIVE'\n                        WHEN MQC.DESCRIPTION =\n                             'FURNITURE, HOME FURNISHINGS AND EQUIPMENT SHOPS AND MANUFACTURERS, EXCEPT APPLIANCES'\n                            THEN 'HOME SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'MISCELLANEOUS PERSONAL SERVICES - NOT ELSEWHERE CLASSIFIED'\n                            THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'AUTOMOTIVE PARTS AND ACCESSORIES OUTLETS' THEN 'AUTOMOTIVE'\n                        WHEN MQC.DESCRIPTION = 'PUBLIC GOLF COURSES' THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'PET SHOPS, PET FOOD AND SUPPLIES' THEN 'PET SHOP'\n                        WHEN MQC.DESCRIPTION = 'MEN’S AND BOYS’ CLOTHING AND ACCESSORY SHOPS' THEN 'CLOTHING FASHION'\n                        WHEN MQC.DESCRIPTION = 'HOME SUPPLY WAREHOUSE OUTLETS' THEN 'HOME SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'HOBBY, TOY AND GAME SHOPS' THEN 'HOBBY, TOY AND GAME SHOPS'\n                        WHEN MQC.DESCRIPTION = 'LAUNDRY SERVICES - FAMILY AND COMMERCIAL' THEN 'LAUNDRY'\n                        WHEN MQC.DESCRIPTION = 'DAIRIES' THEN 'GROCERIES AND SUPERMARKETS'\n                        WHEN MQC.DESCRIPTION = 'SHOE SHOPS' THEN 'CLOTHING FASHION'\n                        WHEN MQC.DESCRIPTION = 'PAYMENT SERVICE PROVIDER' THEN 'TELCO &amp; PEYMENT'\n                        WHEN MQC.DESCRIPTION = 'FLORISTS' THEN 'HOBBY, TOY AND GAME SHOPS'\n                        WHEN MQC.DESCRIPTION = 'LAUNDRY, CLEANING AND GARMENT SERVICES' THEN 'LAUNDRY'\n                        WHEN MQC.DESCRIPTION = 'CANDY, NUT AND CONFECTIONERY SHOPS' THEN 'FOOD &amp; BEVERAGES'\n                        WHEN MQC.DESCRIPTION = 'MISCELLANEOUS APPAREL AND ACCESSORY SHOPS' THEN 'BEAUTY SHOP'\n                        WHEN MQC.DESCRIPTION = 'CONSTRUCTION MATERIALS (NOT ELSEWHERE CLASSIFIED)'\n                            THEN 'BUILDING, ELECTRONICS, AUTOMOTIVE SERVICES'\n                        WHEN MQC.DESCRIPTION = 'BLUEPRINTING AND PHOTOCOPYING SERVICES' THEN 'OFFICE &amp; SCHOOL SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'SCHOOLS AND EDUCATIONAL SERVICES (NOT ELSEWHERE CLASSIFIED)'\n                            THEN 'OFFICE &amp; SCHOOL SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'LODGING - HOTELS, MOTELS AND RESORTS' THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'GIFT, CARD, NOVELTY AND SOUVENIR SHOPS' THEN 'BEAUTY SHOP'\n                        WHEN MQC.DESCRIPTION = 'HEALTH AND BEAUTY SPAS' THEN 'BEAUTY AND BARBER SHOPS'\n                        WHEN MQC.DESCRIPTION = 'DRUG STORES AND PHARMACIES' THEN 'HOSPITAL &amp; MEDICAL'\n                        WHEN MQC.DESCRIPTION = 'INDUSTRIAL SUPPLIES (NOT ELSEWHERE CLASSIFIED)'\n                            THEN 'BUILDING, ELECTRONICS, AUTOMOTIVE SERVICES'\n                        WHEN MQC.DESCRIPTION = 'JEWELLERY, WATCH, CLOCK AND SILVERWARE SHOPS' THEN 'CLOTHING FASHION'\n                        WHEN MQC.DESCRIPTION = 'AQUARIUMS, SEAQUARIUMS AND DOLPHINARIUMS'\n                            THEN 'HOBBY, TOY AND GAME SHOPS'\n                        WHEN MQC.DESCRIPTION = 'WHOLESALE CLUBS' THEN 'WHOLESALE CLUBS'\n                        WHEN MQC.DESCRIPTION = 'AUTOMOTIVE BODY REPAIR SHOPS' THEN 'AUTOMOTIVE'\n                        WHEN MQC.DESCRIPTION = 'DENTAL/LABORATORY/MEDICAL/OPHTHALMIC HOSPITAL EQUIPMENT AND SUPPLIES'\n                            THEN 'HOSPITAL &amp; MEDICAL'\n                        WHEN MQC.DESCRIPTION = 'STATIONERY, OFFICE SUPPLIES AND PRINTING AND WRITING PAPER'\n                            THEN 'OFFICE &amp; SCHOOL SUPPLIES'\n                        WHEN MQC.DESCRIPTION =\n                             'TELECOMMUNICATION SERVICES, INCLUDING LOCAL AND LONG DISTANCE CALLS, CREDIT CARD CALLS, CALLS THROUGH USE OF MAGNETIC STRIPE READING TELEPHONES AND FAXES'\n                            THEN 'TELECOMMUNICATION EQUIPMENT AND TELEPHONE SALES'\n                        WHEN MQC.DESCRIPTION = 'OPTICAL GOODS AND EYEGLASSES' THEN 'BEAUTY SHOP'\n                        WHEN MQC.DESCRIPTION = 'CAR WASHES' THEN 'AUTOMOTIVE'\n                        WHEN MQC.DESCRIPTION = 'CHILDREN’S AND INFANTS’ WEAR SHOPS' THEN 'CLOTHING FASHION'\n                        WHEN MQC.DESCRIPTION = 'FLORISTS, SUPPLIES, NURSERY STOCK AND FLOWERS' THEN 'HOME SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'REAL ESTATES AGENT, MANAGER &amp; RENTAL'\n                            THEN 'BUILDING, ELECTRONICS, AUTOMOTIVE SERVICES'\n                        WHEN MQC.DESCRIPTION = 'CIGAR SHOPS AND STANDS' THEN 'HOBBY, TOY AND GAME SHOPS'\n                        WHEN MQC.DESCRIPTION = 'UTILITIES-ELECTRIC, GAS, WATER AND SANITARY' THEN 'HOME SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'RELIGIOUS ORGANIZATIONS' THEN 'RELIGIOUS ORGANIZATIONS'\n                        WHEN MQC.DESCRIPTION = 'Religious Organization' THEN 'RELIGIOUS ORGANIZATIONS'\n                        WHEN MQC.DESCRIPTION = 'MISCELLANEOUS GENERAL MERCHANDISE' THEN 'RETAIL OUTLETS'\n                        WHEN MQC.DESCRIPTION = 'MOTOR VEHICLE SUPPLIES AND NEW PARTS' THEN 'AUTOMOTIVE'\n                        WHEN MQC.DESCRIPTION = 'ARTIST SUPPLY AND CRAFT SHOPS' THEN 'BEAUTY SHOP'\n                        WHEN MQC.DESCRIPTION = 'PARKING LOTS AND GARAGES'\n                            THEN 'BUILDING, ELECTRONICS, AUTOMOTIVE SERVICES'\n                        WHEN MQC.DESCRIPTION = 'MEDICAL SERVICES AND HEALTH PRACTITIONERS (NOT ELSEWHERE CLASSIFIED)'\n                            THEN 'HOSPITAL &amp; MEDICAL'\n                        WHEN MQC.DESCRIPTION = 'ADVERTISING SERVICES' THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'AIR CONDITIONING AND REFRIGERATION REPAIR SHOPS' THEN 'HOME SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'STATIONERY, OFFICE AND SCHOOL SUPPLY SHOPS'\n                            THEN 'OFFICE &amp; SCHOOL SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'HOUSEHOLD APPLIANCE SHOPS' THEN 'HOME SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'AIRPORTS, FLYING FIELDS AND AIRPORT TERMINALS' THEN 'OTHER SERVICES'\n                        WHEN MQC.DESCRIPTION = 'ART DEALERS AND GALLERIES' THEN 'HOBBY, TOY AND GAME SHOPS'\n                        WHEN MQC.DESCRIPTION = 'MOTORCYCLE SHOPS AND DEALERS' THEN 'AUTOMOTIVE'\n                        WHEN MQC.DESCRIPTION = 'HARDWARE SHOPS' THEN 'HOME SUPPLIES'\n                        WHEN MQC.DESCRIPTION = 'MUSIC SHOPS - MUSICAL INSTRUMENTS, PIANOS AND SHEET MUSIC'\n                            THEN 'HOBBY, TOY AND GAME SHOPS'\n                        WHEN MQC.DESCRIPTION = 'SPORTING GOODS SHOPS' THEN 'HOBBY, TOY AND GAME SHOPS'\n                        WHEN MQC.DESCRIPTION = 'MOTOR HOME AND RECREATIONAL VEHICLE RENTALS' THEN 'AUTOMOTIVE'\n                        WHEN MQC.DESCRIPTION = 'ELECTRONICS REPAIR SHOPS'\n                            THEN 'BUILDING, ELECTRONICS, AUTOMOTIVE SERVICES'\n                        WHEN MQC.DESCRIPTION = 'COURIER SERVICES-AIR AND GROUND AND FREIGHT FORWARDERS'\n                            THEN 'BUILDING, ELECTRONICS, AUTOMOTIVE SERVICES'\n                        WHEN MQC.DESCRIPTION = 'SERVICE STATIONS (WITH OR WITHOUT ANCILLARY SERVICES)'\n                            THEN 'BUILDING, ELECTRONICS, AUTOMOTIVE SERVICES'\n                        ELSE 'OTHERS' END          AS MERCHANT_CATEGORY\n</code></pre>\n<p> </p>\n",
        "tanggal_content": "2023-05-03T07:30:34.501Z",
        "author_content": "Rahmi",
        "direktorat_content": "Digital Insight",
        "PIC_Analyst": "Rahmi Julianasari",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/analisis_merchant_category"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#agi-new-mobile-apps-ux-research\"></a> AGI New Mobile Apps - UX Research</h1>",
        "text_content": "<h1><a href=\"#agi-new-mobile-apps-ux-research\"></a> AGI New Mobile Apps - UX Research</h1>\n<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<th>Direktorat</th>\n<td> Digital Insight</td>\n</tr>\n<tr>\n<th>Sub Direktorat</th>\n<td>MIS</td>\n</tr>\n<tr>\n<th>Analysis Date</th>\n<td>7 April 2022</td>\n</tr>\n<tr>\n<th>Analysis status</th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>Document status </th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>PIC  Analyst</th>\n<td> Rahmi Julianasari</td>\n</tr>\n<tr>\n<th>Stakeholders</th>\n<td>DIgital Banking</td>\n</tr>\n<tr>\n<th>Reference Document</th>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p><span><strong>Version History</strong></span></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><i><strong>Version</strong></i></td>\n<td><i><strong>Date</strong></i></td>\n<td><i><strong>Description</strong></i></td>\n</tr>\n<tr>\n<td> 1.0</td>\n<td>2022-04-07 </td>\n<td>initial document</td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#usability-test-plan-scenario\"></a> Usability Test Plan: Scenario</h2>\n<p><strong>User Goal</strong></p>\n<ol>\n<li>User melakukan registrasi</li>\n<li>User melakukan login</li>\n<li>User melakukan transaksi</li>\n</ol>\n<p><strong>Scenario</strong></p>\n<ol>\n<li>User “baru” melakukan registrasi</li>\n<li>User “baru” melakukan login</li>\n<li>User “baru” melakukan transaksi <strong>transfer</strong></li>\n<li>User “baru” melakukan transaksi <strong>QRIS</strong></li>\n<li>User “baru” melakukan transaksi “<strong>Pembayaran X</strong>”</li>\n<li>User “baru” melakukan transaksi “<strong>Pembelian A</strong>”</li>\n<li>User “baru” melakukan transaksi “<strong>Top Up P</strong>”</li>\n</ol>\n<h3><a href=\"#research-feedback\"></a> Research + feedback</h3>\n<figure><img src=\"/aginewmobile/untitled.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_1.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_2.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_3.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_4.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_5.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_6.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_7.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_8.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_9.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_10.png\"/></figure>\n<figure><img src=\"/aginewmobile/untitled_11.png\"/></figure>\n<p> </p>\n<p> </p>\n<h3><a href=\"#discovered-problem\"></a> Discovered problem</h3>\n<p><i>“categorize the pain points from Empathize”</i></p>\n<p>Instruksi belum cukup clear dan penempatan instruksi tidak sesuai</p>\n<p><strong>Reg 2-4, CA 1-2</strong></p>\n<p>Beberapa fitur icon masih ada yang membingungkan untuk apa</p>\n<p><strong>Reg 1, O 1, O 5, O 6, T 1, PP 1,</strong></p>\n<p>Redundansi menu, tidak efisien</p>\n<p><strong>O2, T2, PB1, TOP1,</strong></p>\n<p>Fitur transaksi banyak yang ngumpet</p>\n<p><strong>O3, O4</strong></p>\n<p>Ada step yang ga terduga perlu dilakukan</p>\n<p><strong>PB 1, TOP 1</strong></p>\n<h3><a href=\"#ideation-suggest-for-improvement\"></a> Ideation (suggest for improvement)</h3>\n<p><strong>Instruksi belum cukup clear dan penempatan instruksi tidak sesuai</strong></p>\n<ol>\n<li>Memberikan on boarding, dengan 2 case</li>\n</ol>\n<ul>\n<li>Account baru</li>\n<li>Baru menginstall regardless akun baru atau bukan</li>\n</ul>\n<figure><img src=\"/aginewmobile/untitled_12.png\"/></figure>\n<p>2. Pada Registrasi foto KTP diberi petunjuk “Ambil foto KTP”</p>\n<p>3. Pada Registrasi foto KTP diberi petunjuk contoh gambar hasil foto KTP yang benar dan salah</p>\n<p><i>ini fungsinya biar user ngerti harus kaya gimana kalo foto KTP yang benar</i></p>\n<p>point 2 dan 3 berlaku untuk take selfie dengan KTP</p>\n<p><strong>for example:</strong></p>\n<figure><img src=\"/aginewmobile/untitled_13.png\"/></figure>\n<p>Screen take foto KTP diperbaiki box-nya; 1 box KTP aja</p>\n<figure><img src=\"/aginewmobile/untitled_6.png\"/></figure>\n<p><strong>Beberapa fitur icon masih ada yang membingungkan untuk apa</strong></p>\n<p>belum kepikiran, mungkin onboarding atau dari penempatan di step yang mana</p>\n<p><strong>Redundansi menu, tidak efisien</strong></p>\n<p>All Fav menu bisa di ganti dengan fitur transaksi lain saja a.k.a dihilangkan</p>\n<p>“Transfer from” dan “source of fund” di merge?? Pas pilih “Transfer from” drop down “source of fund”</p>\n<p>PB 1 dan TOP 1 dibuat 1 page aja; di atas section menu dna di bawah section transaction history</p>\n<ol>\n<li>opsi lain transction history di bikin icon sendiri aja di pojok kanan atas</li>\n</ol>\n<p><strong>Fitur transaksi banyak yang ngumpet</strong></p>\n<ol>\n<li>menampilkan beberapa fitur utama di halaman depan<ol>\n<li>bisa by favorite, most transaction used (personalised); tapi ini butuh default menu</li>\n<li>bisa ditentukan oleh kita berdasarkan data</li>\n</ol>\n</li>\n</ol>\n<p><strong>Ada step yang ga terduga perlu dilakukan</strong></p>\n",
        "tanggal_content": "2022-12-28T10:30:54.097Z",
        "author_content": "Rahmi",
        "direktorat_content": "Digital Insight",
        "PIC_Analyst": "Rahmi Julianasari",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/AGINewMobileAppsUXResearch"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#analisis-kuadran-merchant-berdasarkan-transaksi-balance-dan-amount\"></a> Analisis Kuadran Merchant berdasarkan Transaksi, Balance, dan Amount</h1>",
        "text_content": "<h1><a href=\"#analisis-kuadran-merchant-berdasarkan-transaksi-balance-dan-amount\"></a> Analisis Kuadran Merchant berdasarkan Transaksi, Balance, dan Amount</h1>\n<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<th>Direktorat</th>\n<td> Digital Insight</td>\n</tr>\n<tr>\n<th>Sub Direktorat</th>\n<td>MIS 1 </td>\n</tr>\n<tr>\n<th>Analysis Date</th>\n<td>28 Desember 2022</td>\n</tr>\n<tr>\n<th>Analysis status</th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>Document status </th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>PIC  Analyst</th>\n<td>Rahmi Julianasari</td>\n</tr>\n<tr>\n<th>Stakeholders</th>\n<td>DIgital Banking</td>\n</tr>\n<tr>\n<th>Reference Document</th>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#version-history\"></a> Version History</h2>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><i><strong>Version</strong></i></td>\n<td><i><strong>Date</strong></i></td>\n<td><i><strong>Description</strong></i></td>\n</tr>\n<tr>\n<td> 1.0</td>\n<td>2022-04-07 </td>\n<td>initial document</td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#latar-belakang-dan-tujuan\"></a> Latar Belakang dan Tujuan</h2>\n<p>Marketing analytics merupakan salah satu target pada RBB 2023. Tujuan analisis adalah menentukan target customer untuk ditawarkan produk BAG yang sesuai dengan karakteristik customer berdasarkan hasil analisis. Pada analisis ini, penentuan target customer dilakukan pada Merchant untuk penawaran produk SME dan Konsumer. Data-data seperti transaksi dan balance rekening perlu dilakukan untuk memberikan gambaran potensial merchant.</p>\n<p> </p>\n<h2><a href=\"#analisis-populasi-data-merchant\"></a> Analisis Populasi Data Merchant</h2>\n<p>Terdapat beberapa hal yang dianalisis untuk mendapatkan populasi merchant yang akan ditawarkan produk BAG.</p>\n<ol>\n<li>Merchant bukan merupakan sharing account yang dikelola oleh BAG atau cabang</li>\n<li>Merchant bukan merupakan sharing account dari perusahaan yang ada di grup</li>\n<li>Merchant memiliki transaksi minimal 1 sejak 90 hari terakhir (data per 28/12/2022)</li>\n</ol>\n<p>Berikut ringkasan hasil analisis populasi merchant</p>\n<ul>\n<li>Total Account Number = 3179</li>\n<li>Total Merchant = 3335</li>\n</ul>\n<p>Data Merchant yang dikumpulkan</p>\n<ul>\n<li>Account Number</li>\n<li>Jumlah Merchant per Account Number</li>\n<li>Total Transaksi dalah 90 hari terakhir</li>\n<li>Total Amount Transaksi dalam 90 hari terakhir</li>\n<li>Average balance 90 hari terakhir</li>\n</ul>\n<p><i>*) data per 28/12/2022</i></p>\n<p> </p>\n<h2><a href=\"#analisis-kuadran-transaksi-dan-balance\"></a> Analisis Kuadran Transaksi dan Balance</h2>\n<p>Beberapa simulasi dilakukan dengan mencari proporsi Top X% merchant dengan transaksi tertinggi dan Top Y% merchant dengan balance tertinggi.</p>\n<p> </p>\n<figure><img src=\"/insight/trx_vs_bal.jpg\"/></figure>\n<p> </p>\n<h3><a href=\"#simulasi-1-25-transaksi-vs-25-balance\"></a> <mark>Simulasi 1 (25% Transaksi vs 25% Balance)</mark></h3>\n<p>Dengan simulasi pertama, menggunakan TOP 25% merchant dengan transaksi terbanyak dan TOP 25% merchant dengan balance tertinggi. Diperoleh jumlah account per kuadran sebagai berikut:</p>\n<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>2</td>\n</tr>\n<tr>\n<td>II</td>\n<td>16</td>\n</tr>\n<tr>\n<td>III</td>\n<td>3146</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>15</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<h3><a href=\"#simulasi-2-50-transaksi-vs-50-balance\"></a> <mark>Simulasi 2 (50% Transaksi vs 50% Balance)</mark></h3>\n<p>Dengan simulasi pertama, menggunakan TOP 50% merchant dengan transaksi terbanyak dan TOP 50% merchant dengan balance tertinggi. Diperoleh jumlah account per kuadran sebagai berikut:</p>\n<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>6</td>\n</tr>\n<tr>\n<td>II</td>\n<td>67</td>\n</tr>\n<tr>\n<td>III</td>\n<td>3062</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>44</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<h3><a href=\"#simulasi-3-proporsional\"></a> <mark>Simulasi 3 (proporsional)</mark></h3>\n<p><strong>Statistik Data Transaksi Merchant</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><strong>Kalkulasi</strong></td>\n<td><strong>TRX</strong></td>\n<td><strong> posisi % </strong></td>\n</tr>\n<tr>\n<td>Average Transaksi</td>\n<td>147</td>\n<td rowspan=\"5\">\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n</td>\n</tr>\n<tr>\n<td>P25</td>\n<td>3</td>\n</tr>\n<tr>\n<td>P50</td>\n<td>12</td>\n</tr>\n<tr>\n<td>P75</td>\n<td>63</td>\n</tr>\n<tr>\n<td>P90</td>\n<td>272</td>\n</tr>\n<tr>\n<td><strong>P95</strong></td>\n<td><strong>604</strong></td>\n<td><strong>66%</strong></td>\n</tr>\n<tr>\n<td><strong>P97</strong></td>\n<td><strong>1021</strong></td>\n<td><strong>60%</strong></td>\n</tr>\n<tr>\n<td><strong>P99</strong></td>\n<td><strong>2615,8</strong></td>\n<td><strong>34%</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p><strong>Statistik Data Merchant Balance</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><strong>Kalkulasi</strong></td>\n<td><strong>Balance</strong></td>\n<td><strong> posisi % </strong></td>\n</tr>\n<tr>\n<td>Average Balance</td>\n<td> 7.528.943 </td>\n<td rowspan=\"5\">\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n</td>\n</tr>\n<tr>\n<td>P25</td>\n<td>           49.617 </td>\n</tr>\n<tr>\n<td>P50</td>\n<td>        260.963 </td>\n</tr>\n<tr>\n<td>P75</td>\n<td>     1.531.041 </td>\n</tr>\n<tr>\n<td>P90</td>\n<td> 10.363.817 </td>\n</tr>\n<tr>\n<td><strong>P95</strong></td>\n<td><strong>   30.824.641 </strong></td>\n<td><strong>77%</strong></td>\n</tr>\n<tr>\n<td><strong>P97</strong></td>\n<td><strong>   61.359.130 </strong></td>\n<td><strong>65%</strong></td>\n</tr>\n<tr>\n<td><strong>P99</strong></td>\n<td><strong> 192.746.213 </strong></td>\n<td><strong>38%</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p>Mengacu pada data statistik di atas, secara “expert judgment” angka transaksi yang di dalam batas normal ada pada posisi P95 (66%),  P97 (60%) atau P99 (34%). Untuk balance, batas normal berada pada posisi P95 (77%),  P97 (65) atau P99 (38%). </p>\n<p>Simulasi untuk <strong>Transaksi P95</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td colspan=\"2\">\n<p><strong>Balance P95</strong></p>\n</td>\n<td> </td>\n<td colspan=\"2\">\n<p><strong>Balance P97</strong></p>\n</td>\n<td>\n<p> </p>\n</td>\n<td colspan=\"2\">\n<p><strong>Balance P99</strong></p>\n</td>\n</tr>\n<tr>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>33</td>\n<td> </td>\n<td>I</td>\n<td>20</td>\n<td> </td>\n<td>I</td>\n<td>7</td>\n</tr>\n<tr>\n<td>II</td>\n<td>125</td>\n<td> </td>\n<td>II</td>\n<td>138</td>\n<td> </td>\n<td>II</td>\n<td>151</td>\n</tr>\n<tr>\n<td>III</td>\n<td>2896</td>\n<td> </td>\n<td>III</td>\n<td>2948</td>\n<td> </td>\n<td>III</td>\n<td>2997</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>125</td>\n<td> </td>\n<td>IV</td>\n<td>73</td>\n<td> </td>\n<td>IV</td>\n<td>24</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p>Simulasikan untuk <strong>Transaksi P97</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td colspan=\"2\">\n<p><strong>Balance P95</strong></p>\n</td>\n<td> </td>\n<td colspan=\"2\">\n<p><strong>Balance P97</strong></p>\n</td>\n<td>\n<p> </p>\n</td>\n<td colspan=\"2\">\n<p><strong>Balance P99</strong></p>\n</td>\n</tr>\n<tr>\n<td>Kuadran</td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>28</td>\n<td> </td>\n<td>I</td>\n<td>15</td>\n<td> </td>\n<td>I</td>\n<td>4</td>\n</tr>\n<tr>\n<td>II</td>\n<td>90</td>\n<td> </td>\n<td>II</td>\n<td>103</td>\n<td> </td>\n<td>II</td>\n<td>114</td>\n</tr>\n<tr>\n<td>III</td>\n<td>2931</td>\n<td> </td>\n<td>III</td>\n<td>2983</td>\n<td> </td>\n<td>III</td>\n<td>3034</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>130</td>\n<td> </td>\n<td>IV</td>\n<td>78</td>\n<td> </td>\n<td>IV</td>\n<td>27</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p>Simulasi untuk <strong>Transaksi P99</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td colspan=\"2\">\n<p><strong>Balance P95</strong></p>\n</td>\n<td> </td>\n<td colspan=\"2\">\n<p><strong>Balance P97</strong></p>\n</td>\n<td>\n<p> </p>\n</td>\n<td colspan=\"2\">\n<p><strong>Balance P99</strong></p>\n</td>\n</tr>\n<tr>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>15</td>\n<td> </td>\n<td>I</td>\n<td>7</td>\n<td> </td>\n<td>I</td>\n<td>2</td>\n</tr>\n<tr>\n<td>II</td>\n<td>16</td>\n<td> </td>\n<td>II</td>\n<td>24</td>\n<td> </td>\n<td>II</td>\n<td>29</td>\n</tr>\n<tr>\n<td>III</td>\n<td>3005</td>\n<td> </td>\n<td>III</td>\n<td>3062</td>\n<td> </td>\n<td>III</td>\n<td>3119</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>143</td>\n<td> </td>\n<td>IV</td>\n<td>86</td>\n<td> </td>\n<td>IV</td>\n<td>29</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<h2><a href=\"#analisis-kuadran-transaksi-dan-amount\"></a> Analisis Kuadran Transaksi dan Amount</h2>\n<p>Beberapa simulasi dilakukan dengan mencari proporsi Top X% merchant dengan transaksi tertinggi dan Top Y% merchant dengan amount tertinggi.</p>\n<p> </p>\n<figure><img src=\"/insight/trx_x_amt.jpg\"/></figure>\n<p> </p>\n<h3><a href=\"#simulasi-1-25-transaksi-vs-25-amount\"></a> <mark>Simulasi 1 (25% Transaksi vs 25% Amount)</mark></h3>\n<p>Dengan simulasi pertama, menggunakan TOP 25% merchant dengan transaksi terbanyak dan TOP 25% merchant dengan total amount tertinggi. Diperoleh jumlah account per kuadran sebagai berikut:</p>\n<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>7</td>\n</tr>\n<tr>\n<td>II</td>\n<td>3155</td>\n</tr>\n<tr>\n<td>III</td>\n<td>6</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>11</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<h3><a href=\"#simulasi-2-50-transaksi-vs-50-amount\"></a> <br/><mark><strong>Simulasi 2 (50% Transaksi vs 50% Amount)</strong></mark></h3>\n<p>Dengan simulasi pertama, menggunakan TOP 50% merchant dengan transaksi terbanyak dan TOP 50% merchant dengan total amount tertinggi. Diperoleh jumlah account per kuadran sebagai berikut:</p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>36</td>\n</tr>\n<tr>\n<td>II</td>\n<td>3073</td>\n</tr>\n<tr>\n<td>III</td>\n<td>33</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>37</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<h3><a href=\"#simulasi-3-proporsional-1\"></a> <mark>Simulasi 3 (proporsional)</mark></h3>\n<p><strong>Statistik Data Transaksi Merchant</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><strong>Kalkulasi</strong></td>\n<td><strong>TRX</strong></td>\n<td><strong> posisi % </strong></td>\n</tr>\n<tr>\n<td>Average Transaksi</td>\n<td>147</td>\n<td rowspan=\"5\">\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n</td>\n</tr>\n<tr>\n<td>P25</td>\n<td>3</td>\n</tr>\n<tr>\n<td>P50</td>\n<td>12</td>\n</tr>\n<tr>\n<td>P75</td>\n<td>63</td>\n</tr>\n<tr>\n<td>P90</td>\n<td>272</td>\n</tr>\n<tr>\n<td><strong>P95</strong></td>\n<td><strong>604</strong></td>\n<td><strong>66%</strong></td>\n</tr>\n<tr>\n<td><strong>P97</strong></td>\n<td><strong>1021</strong></td>\n<td><strong>60%</strong></td>\n</tr>\n<tr>\n<td><strong>P99</strong></td>\n<td><strong>2615,8</strong></td>\n<td><strong>34%</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p><strong>Statistik Data Merchant Balance</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><strong>Kalkulasi</strong></td>\n<td><strong>Amount</strong></td>\n<td><strong> posisi % </strong></td>\n</tr>\n<tr>\n<td>Average Balance</td>\n<td>   12.532.268 </td>\n<td> </td>\n</tr>\n<tr>\n<td>P25</td>\n<td>       120.000 </td>\n<td> </td>\n</tr>\n<tr>\n<td>P50</td>\n<td>       900.000 </td>\n<td> </td>\n</tr>\n<tr>\n<td>P75</td>\n<td>     5.330.000 </td>\n<td> </td>\n</tr>\n<tr>\n<td>P90</td>\n<td>   23.140.000 </td>\n<td> </td>\n</tr>\n<tr>\n<td><strong>P95</strong></td>\n<td><strong>   53.719.000 </strong></td>\n<td><strong>66%</strong></td>\n</tr>\n<tr>\n<td><strong>P97</strong></td>\n<td><strong>   83.855.680 </strong></td>\n<td><strong>56%</strong></td>\n</tr>\n<tr>\n<td><strong>P99</strong></td>\n<td><strong> 191.971.546 </strong></td>\n<td><strong>37%</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p>Mengacu pada data statistik di atas, secara “expert judgment” angka transaksi yang di dalam batas normal ada pada posisi P95 (66%),  P97 (60%) atau P99 (34%). Untuk balance, batas normal berada pada posisi P95 (66%),  P97 (56%) atau P99 (37%). </p>\n<p>Simulasi untuk <strong>Transaksi P95</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td colspan=\"2\">\n<p><strong>Balance P95</strong></p>\n</td>\n<td> </td>\n<td colspan=\"2\">\n<p><strong>Balance P97</strong></p>\n</td>\n<td>\n<p> </p>\n</td>\n<td colspan=\"2\">\n<p><strong>Balance P99</strong></p>\n</td>\n</tr>\n<tr>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>87</td>\n<td> </td>\n<td>I</td>\n<td>59</td>\n<td> </td>\n<td>I</td>\n<td>28</td>\n</tr>\n<tr>\n<td>II</td>\n<td>2955</td>\n<td> </td>\n<td>II</td>\n<td>2985</td>\n<td> </td>\n<td>II</td>\n<td>3017</td>\n</tr>\n<tr>\n<td>III</td>\n<td>66</td>\n<td> </td>\n<td>III</td>\n<td>36</td>\n<td> </td>\n<td>III</td>\n<td>4</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>71</td>\n<td> </td>\n<td>IV</td>\n<td>99</td>\n<td> </td>\n<td>IV</td>\n<td>130</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p>Simulasikan untuk <strong>Transaksi P97</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td colspan=\"2\">\n<p><strong>Balance P95</strong></p>\n</td>\n<td> </td>\n<td colspan=\"2\">\n<p><strong>Balance P97</strong></p>\n</td>\n<td>\n<p> </p>\n</td>\n<td colspan=\"2\">\n<p><strong>Balance P99</strong></p>\n</td>\n</tr>\n<tr>\n<td>Kuadran</td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>77</td>\n<td> </td>\n<td>I</td>\n<td>57</td>\n<td> </td>\n<td>I</td>\n<td>27</td>\n</tr>\n<tr>\n<td>II</td>\n<td>2985</td>\n<td> </td>\n<td>II</td>\n<td>3023</td>\n<td> </td>\n<td>II</td>\n<td>3056</td>\n</tr>\n<tr>\n<td>III</td>\n<td>76</td>\n<td> </td>\n<td>III</td>\n<td>38</td>\n<td> </td>\n<td>III</td>\n<td>5</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>41</td>\n<td> </td>\n<td>IV</td>\n<td>61</td>\n<td> </td>\n<td>IV</td>\n<td>91</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p>Simulasi untuk <strong>Transaksi P99</strong></p>\n<figure>\n<table>\n<tbody>\n<tr>\n<td colspan=\"2\">\n<p><strong>Balance P95</strong></p>\n</td>\n<td> </td>\n<td colspan=\"2\">\n<p><strong>Balance P97</strong></p>\n</td>\n<td>\n<p> </p>\n</td>\n<td colspan=\"2\">\n<p><strong>Balance P99</strong></p>\n</td>\n</tr>\n<tr>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n<td> </td>\n<td><strong>Kuadran</strong></td>\n<td><strong>Count of ACCOUNT_NUMBER</strong></td>\n</tr>\n<tr>\n<td>I</td>\n<td>30</td>\n<td> </td>\n<td>I</td>\n<td>26</td>\n<td> </td>\n<td>I</td>\n<td>16</td>\n</tr>\n<tr>\n<td>II</td>\n<td>3025</td>\n<td> </td>\n<td>II</td>\n<td>3079</td>\n<td> </td>\n<td>II</td>\n<td>3132</td>\n</tr>\n<tr>\n<td>III</td>\n<td>123</td>\n<td> </td>\n<td>III</td>\n<td>69</td>\n<td> </td>\n<td>III</td>\n<td>16</td>\n</tr>\n<tr>\n<td>IV</td>\n<td>1</td>\n<td> </td>\n<td>IV</td>\n<td>5</td>\n<td> </td>\n<td>IV</td>\n<td>15</td>\n</tr>\n<tr>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n<td> </td>\n<td><strong>Grand Total</strong></td>\n<td><strong>3179</strong></td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<h2><a href=\"#rekomendasi\"></a> Rekomendasi</h2>\n<h3><a href=\"#rekomendasi-1-akuisisi-sme\"></a> <mark>Rekomendasi 1: Akuisisi SME</mark></h3>\n<p>Mengacu pada <strong>Analisis Kuadran Transaksi-Balance, Kuadran I dan IV dengan titik Balance pada P95 dapat dijadikan target untuk AKUISISI SME</strong>. Justifikasinya adalah merchant-merchant pada titik tersebut memiliki Avaragae Balance di batas atas normal (P95). Dengan kata lain merchant yang merupakan nasabah cukup …(istilahnya apa lupa gw) sehingga dapat ditawarkan produk SME.</p>\n<p> </p>\n<h3><a href=\"#rekomendasi-2-melakukan-deep-analysis-untuk-penawaran-produk-konsumer\"></a> <mark>Rekomendasi 2: Melakukan deep analysis untuk penawaran produk Konsumer</mark></h3>\n<p>Mengacu pada <strong>Analisis Kuadran Transaksi-Balance, Kuadran II dengan titik Balance pada P95 dapat dijadikan target untuk AKUISISI KONSUMER</strong>.</p>\n<p> </p>\n<h3><a href=\"#\"></a>  </h3>\n",
        "tanggal_content": "2022-12-30T10:23:27.674Z",
        "author_content": "Rahmi",
        "direktorat_content": "Digital Insight",
        "PIC_Analyst": "Rahmi Julianasari",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/Analisis_Kuadran_Merchant"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#analisis-transaksi-qris-biller\"></a> Analisis Transaksi Qris &amp; Biller</h1>",
        "text_content": "<h1><a href=\"#analisis-transaksi-qris-biller\"></a> Analisis Transaksi Qris &amp; Biller</h1>\n<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<th>Direktorat</th>\n<td> Digital Insight</td>\n</tr>\n<tr>\n<th>Sub Direktorat</th>\n<td>MIS 1 </td>\n</tr>\n<tr>\n<th>Analysis Date</th>\n<td>28 Desember 2022</td>\n</tr>\n<tr>\n<th>Analysis status</th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>Document status </th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>PIC  Analyst</th>\n<td>Ghifari Daulagiri</td>\n</tr>\n<tr>\n<th>Stakeholders</th>\n<td>DIgital Banking</td>\n</tr>\n<tr>\n<th>Reference Document</th>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#version-history\"></a> Version History</h2>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><i><strong>Version</strong></i></td>\n<td><i><strong>Date</strong></i></td>\n<td><i><strong>Description</strong></i></td>\n</tr>\n<tr>\n<td> 1.0</td>\n<td>2022-04-07 </td>\n<td>initial document</td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#latar-belakang-dan-tujuan\"></a> LATAR BELAKANG DAN TUJUAN</h2>\n<p>Analisis ini dibuat untuk memberikan insight dalam penentuan fokus dan opsi untuk menentukan promo yang akan diberikan untuk menstimulasi proses transaksi pada merchant yang dituju</p>\n<p> </p>\n<h2><a href=\"#temuan-utama\"></a> Temuan Utama</h2>\n<p>Data yang digunakan adalah dana transasksi QRIS dari Bulan Maret 2021 dan dana transaksi Pembelian dan Pembayaran dari Tahun 2017</p>\n<h3><a href=\"#biller\"></a> BILLER</h3>\n<p>Detail Isi Pulsa</p>\n<p>List Provider</p>\n<p> </p>\n<p>Data Di atas merupakan data dari banyaknya transaksi dan besar transaksi yang dilakukan oleh nasabah ke provider</p>\n<blockquote>\n<p>Dari data diatas kita bisa lihat bahwa transaksi terbanyak Dilakukan provider Telkomsel bahkan jumlahnya melebihi semua provider lain jika digabungkan.</p>\n</blockquote>\n<p>Banyak Transaksi</p>\n<p> </p>\n<p>Ringkasan Perbulan</p>\n<p>Berikut merupakan ringkasan jumlah transaksi dan besar nilai transaksi yang dilakukan nasabah</p>\n<p>Maret 22</p>\n<p> </p>\n<p>Febuari 22</p>\n<p> </p>\n<p>Januari 22</p>\n<p> </p>\n<p>Desember 21</p>\n<p> </p>\n<p>November 21</p>\n<p> </p>\n<p>Oktober 21</p>\n<p> </p>\n<p>September 21</p>\n<p> </p>\n<h2><a href=\"#kesimpulan\"></a> Kesimpulan</h2>\n<p>Ada beberapa hal yang perlu digarisbawahi:</p>\n<ol>\n<li>Terdapat appetite untuk menggunakan platform BAG dalam melakukan pembayaran</li>\n<li>Pembayaran transaksi Terbanyak dilakukan pada Pembelian Isi Ulang Pulsa selama 6 bulan terakhir</li>\n<li>Tekomsel merupakan provider dengan nilai transaksi terbesar dan jumlah transaksi terbanyak</li>\n</ol>\n<h1><a href=\"#rekomendasi\"></a> Rekomendasi</h1>\n<p>Perlu riset dan survey lebih lanjut ke segmen nasabah tersebut untuk mencari tahu lebih jauh mengenai preferensi transaksi mereka.</p>\n<h1><a href=\"#qris\"></a> QRIS</h1>\n<p>Ringkasan Terbesar</p>\n<p>Berikut merupakan ringkasan dari nilai transaksi dan banyaknya transaksi yang dilakukan dengan nasabah dalam pembayaran QRIS</p>\n<p>10 Merchant Paling banyak Melakukan Transaksi</p>\n<p> </p>\n<p>10 Merchant Dengan Nilai Transaksi Terbesar</p>\n<p> </p>\n<p>Ringkasan Terkecil</p>\n<p>Berikut merupakan ringkasan dari nilai transaksi dan banyaknya transaksi yang dilakukan dengan nasabah dalam pembayaran QRIS</p>\n<p>10 Merchant Paling Sedikit Melakukan Transaksi dengan Nilai Transaksi Sedikit</p>\n<p> </p>\n<p>10 Merchant Paling Sedikit Melakukan Transaksi dengan Nilai Transaksi Paling Besar</p>\n<p> </p>\n<blockquote>\n<p>Data diatas merupakan data dari merchant yang sedikit melakukan transaksi tetapi nilai nilai transaksi besar. merchant dengan data tersebut melakukan satu transaksi dengan nilai transaksi yang besar</p>\n</blockquote>\n<p>Kategori Merchant</p>\n<p>Berikut kategori-kategori merchant yang bekerjasama dengan bank</p>\n<p>Berdasarkan Kategori Merchant</p>\n<p> </p>\n<p>Berdasarkan Kode kategori yang ada</p>\n<p> </p>\n<p>Berdasarkan Badan Usaha</p>\n<p> </p>\n<p>Data Perbulan</p>\n<p>Maret 22</p>\n<p> </p>\n<p>Febuari 22</p>\n<p> </p>\n<p>Januari 22</p>\n<p> </p>\n<p>Desember 21</p>\n<p> </p>\n<p>November 21</p>\n<p> </p>\n<p>Oktober 21</p>\n<p> </p>\n<p>September 21</p>\n<p> </p>\n<p>Data diatas merupakan data dari nilai transaksi dan Banyak transaksi perbulannya selama 6 bulan terakhir</p>\n<h2><a href=\"#kesimpulan-1\"></a> Kesimpulan</h2>\n<p>Ada beberapa hal yang perlu digarisbawahi:</p>\n<ol>\n<li>Terdapat appetite untuk menggunakan platform BAG dalam melakukan pembayaran</li>\n<li>tidak ada merchant yang menempati urutan tertinggi selama 2 bulan dengan nilai transaksi terbesar</li>\n<li>ada merchant yang melakukan nilai transaksi yang besar dengan transaksi yang sedikit</li>\n</ol>\n<h1><a href=\"#rekomendasi-1\"></a> Rekomendasi</h1>\n<p>Untuk rekomendasi pemberian Promo pada merchant kita bisa memberikan beberapa opsi yang bisa menguntungkan pembeli. dari hasil data diatas kita bisa memberikan promo dengan cara:</p>\n<ul>\n<li>Memberikan Jenis Promo untuk barang cepat habis yang bertujuan untuk merchant dengan jumlah transaksi yang banyak. hal ini dapat menstimulasi pembeli pada merchant yang menjual barang yang dapat dikonsumsi dan harganya murah. jenis promo yang diberikan bisa berupa potongan harga atau bonus menu untuk pembelian dengan harga tertentu</li>\n<li>Memberikan Penurunan harga untuk produk tertentu jika sudah mencapai harga yang ditentukan. hal ini dapat menstimulasi pembeli untuk membeli barang jika harga barang yang seharusnya dapat dibeli dengan harga yang lebih murah</li>\n<li>Memberikan undian ketika membeli barang dengan harga tertentu dan berlaku kelipatan</li>\n</ul>\n",
        "tanggal_content": "2022-12-28T08:44:26.858Z",
        "author_content": "Rahmi",
        "direktorat_content": "Digital Insight",
        "PIC_Analyst": "Ghifari Daulagiri",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/AnalisisTransaksiQRISBiller"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#emoney-abuse\"></a> Emoney Abuse</h1>",
        "text_content": "<h1><a href=\"#emoney-abuse\"></a> Emoney Abuse</h1>\n<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<th>Direktorat</th>\n<td> Digital Insight</td>\n</tr>\n<tr>\n<th>Sub Direktorat</th>\n<td>MIS 1 </td>\n</tr>\n<tr>\n<th>Analysis Date</th>\n<td>19 Juni 2022</td>\n</tr>\n<tr>\n<th>Analysis status</th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>Document status </th>\n<td> FINISHED</td>\n</tr>\n<tr>\n<th>PIC  Analyst</th>\n<td>Rahmi Julianasari</td>\n</tr>\n<tr>\n<th>Stakeholders</th>\n<td>DIgital Banking</td>\n</tr>\n<tr>\n<th>Reference Document</th>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#version-history\"></a> Version History</h2>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><i><strong>Version</strong></i></td>\n<td><i><strong>Date</strong></i></td>\n<td><i><strong>Description</strong></i></td>\n</tr>\n<tr>\n<td> 1.0</td>\n<td>19 Juni 2022</td>\n<td>initial document</td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p><a href=\"/meeting_notes.pdf\">/meeting_notes.pdf</a></p>\n<h3><a href=\"#kronologi\"></a> Kronologi</h3>\n<ul>\n<li>Narasi</li>\n</ul>\n<figure>\n<table>\n<thead>\n<tr>\n<th>No</th>\n<th>Korban</th>\n<th>Pelaku</th>\n<th>BAG</th>\n</tr>\n</thead>\n<tbody>\n<tr>\n<td>1</td>\n<td> </td>\n<td>Menghubungi Korban mengaku sebagai karyawan Shopee</td>\n<td> </td>\n</tr>\n<tr>\n<td>2</td>\n<td>Dihubungi Pelaku bahwa Korban mendapat hadiah/cashback senilai (contoh Rp. 2jt)</td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td>3</td>\n<td> </td>\n<td>Meminta Korban download aplikasi Pinjaman Onliner (ex: Kredivo)</td>\n<td> </td>\n</tr>\n<tr>\n<td>4</td>\n<td>Mendowload dan registrasi akun Pinjaman nline</td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td>5</td>\n<td> </td>\n<td>\n<p>Meminta informasi data Pinjol Korban </p>\n<p>- OTP </p>\n<p>- KTP </p>\n<p>- Foto </p>\n<p>- Ibu Kandung </p>\n<p>- Email </p>\n<p>- dll.</p>\n</td>\n<td> </td>\n</tr>\n<tr>\n<td>6</td>\n<td>Memberikan informasi data Pinjol ke Pelaku</td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td>7</td>\n<td> </td>\n<td>Membajak akun Pinjol Korban</td>\n<td> </td>\n</tr>\n<tr>\n<td>8</td>\n<td> </td>\n<td>Membuat trasnaksi pinjaman di aku Pinjol Korban</td>\n<td> </td>\n</tr>\n<tr>\n<td>9</td>\n<td>Mendapat Notifikasi Pinjaman dari aplikasi Pinjol</td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td>10</td>\n<td> </td>\n<td>Membuat akun BAG atas nama Korban</td>\n<td> </td>\n</tr>\n<tr>\n<td>11</td>\n<td> </td>\n<td>Membuat trasnaksi pinjaman di aku Pinjol Korban</td>\n<td> </td>\n</tr>\n<tr>\n<td>12</td>\n<td> </td>\n<td>Membuat transaksi di akun BAG</td>\n<td> </td>\n</tr>\n<tr>\n<td>13</td>\n<td>Membuat laporan ke BAG dan Kepolisian</td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td>14</td>\n<td> </td>\n<td> </td>\n<td>Mendapat Laporan dari Korban</td>\n</tr>\n<tr>\n<td>15</td>\n<td> </td>\n<td> </td>\n<td>Memblokir akun yang terkait Korban</td>\n</tr>\n</tbody>\n</table>\n</figure>\n<h3><a href=\"#data-analysis\"></a> Data Analysis</h3>\n<p><strong>Data Komplain</strong></p>\n<ul>\n<li>Tanggal transaksi terlapor : 11 Agustus 2021 - 19 Juni 2022</li>\n<li>47 nomor rekening terlapor</li>\n<li>Total nominal terlapor : Rp 83,079,000</li>\n</ul>\n<p><strong>Data Suspect</strong></p>\n<ul>\n<li>Tanggal data terdeteksi : 6 Febuari 2022 - 19 Juni 2022</li>\n<li>41 nomor rekening dengan</li>\n<li>Total Nominal suspicious transaction: Rp 72,590,542</li>\n</ul>\n<p><strong>Document</strong></p>\n<p><a href=\"/clean_suspicious_transcation_old_mb_-_20220624.xlsx\">Clean_Suspicious Transcation OLD MB - 20220624.xlsx</a></p>\n<p><a href=\"/kronologi_pengaduan_nasabah_penyalahgunaan_data_bagi_money.xls\">Kronologi pengaduan nasabah penyalahgunaan data BAGI Money.xls</a></p>\n<h3><a href=\"#rekomendasi\"></a> Rekomendasi</h3>\n<p><strong>Limitasi Friction</strong></p>\n<ol>\n<li>Limitasi amount transaksi per har per user</li>\n<li>Limitasi transaksi User Biasa dan User Upgrade</li>\n</ol>\n<p><strong>User Flow Friction</strong></p>\n<ol>\n<li>OTP</li>\n<li>Upgrade Account</li>\n<li>Create Rekening</li>\n</ol>\n<p><strong>Edukasi</strong></p>\n<ol>\n<li>Memberikan onboarding pada aplikasi di entry point tertentu<ol>\n<li>Registrasi Mobile Banking</li>\n<li>Aktivasi Emoney</li>\n</ol>\n</li>\n<li>Menambahkan wording “menjaga keamanan” pada SMS OTP dan Email notif</li>\n<li>Social Media</li>\n<li>Notifikasi himbauan “security awareness” regular</li>\n</ol>\n<p><strong>Hold Pencairan/Transaksi dengan rekomendasi Amount tertentu</strong></p>\n<ul>\n<li>Penambahan bisnis proses id Anti Fraud</li>\n</ul>\n",
        "tanggal_content": "2022-12-29T04:52:30.070Z",
        "author_content": "Rahmi",
        "direktorat_content": "Digital Insight",
        "PIC_Analyst": "Rahmi Julianasari",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/EmoneyAbuse"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#program-promotion-agi-mobile-acqactv\"></a> Program Promotion Agi Mobile ACQ,ACTV</h1>",
        "text_content": "<h1><a href=\"#program-promotion-agi-mobile-acqactv\"></a> Program Promotion Agi Mobile ACQ,ACTV</h1>\n<p>Donasi</p>\n<p>Pembelian Hewa Kurban</p>\n<p>Jual murah</p>\n<p>Produk</p>\n<ul>\n<li>hewan kurban</li>\n<li>bumbu masak</li>\n<li>arang, tusuk sate</li>\n<li>ketupat</li>\n<li>sembako</li>\n</ul>\n<p>Target subjek</p>\n<ul>\n<li>pedagang hewan kurban</li>\n<li>subject untuk trigger donasi</li>\n<li>pedagang bumbu/arang/tusuk sate/ketupat</li>\n<li>Pedagang sembako, jual murah</li>\n</ul>\n",
        "tanggal_content": "2022-12-29T04:43:13.615Z",
        "author_content": "Arum",
        "direktorat_content": "Tidak ada direktorat",
        "PIC_Analyst": "Tidak ada PIC Analyst",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/ProgramPromotionAgiMobileACQ,ACTV"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#startup-kasirpos-research\"></a> Startup Kasir/POS Research</h1>",
        "text_content": "<h1><a href=\"#startup-kasirpos-research\"></a> Startup Kasir/POS Research</h1>\n<p>Source: <a href=\"https://id.techinasia.com/kumpulan-startup-pos-di-indonesia\">https://id.techinasia.com/kumpulan-startup-pos-di-indonesia</a></p>\n<ol>\n<li>Moka</li>\n<li>Pawoon</li>\n<li>Majoo (add.)</li>\n<li>Olsera</li>\n<li>Deal POS</li>\n<li>Omega POS Cloud</li>\n<li>HelloBill</li>\n<li>Nadi POS</li>\n<li>Meeber POS</li>\n<li>Dokuku</li>\n<li>Folio</li>\n<li>Qasir (Add.)</li>\n</ol>\n<h3><a href=\"#agi-use-case\"></a> AGI use case:</h3>\n<ul>\n<li>pake app kasir/POS partner, otomatis dibukain rekening AG</li>\n<li>pembeli di toko bisa terima pembayaran QRIS pake QRISnya AG</li>\n<li>pembeli di toko bisa pakai payment channelnya paprika (<a href=\"https://www.paprika.co.id\">https://www.paprika.co.id</a>)</li>\n<li>prefer yang belum punya partnership eksklusif sama bank lain</li>\n</ul>\n<h3><a href=\"#comparison\"></a> Comparison</h3>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><strong>Startup</strong></td>\n<td><strong>Website</strong></td>\n<td><strong>Tag</strong></td>\n<td><strong>Bisnis Model</strong></td>\n<td><strong>Partnership</strong></td>\n<td><strong>QRIS/VA/Payment Channel Use Case</strong></td>\n<td><strong> </strong></td>\n</tr>\n<tr>\n<td>Moka</td>\n<td><code>https://www.mokapos.com/</code></td>\n<td>\n<p>- POS </p>\n<p>- CRM </p>\n<p>- Digital Lending (Moka Capital)</p>\n</td>\n<td>\n<p>- Tablet is installed Moka POS mobile app </p>\n<p>- Hardware<i> </i></p>\n<p><i>- S</i>ubscription </p>\n<p>- Moka Capital (digital lending) </p>\n<p>- Digital Payment</p>\n</td>\n<td>\n<p>- PT Mandiri Capital Indonesia, Bank Mandiri</p>\n<p> - Modalku </p>\n<p>- Commonwealth</p>\n</td>\n<td>\n<p>- e-Wallet (digital payment) </p>\n<p>- QRIS *MDR is applied</p>\n</td>\n<td> </td>\n</tr>\n<tr>\n<td>Pawoon</td>\n<td><code>https://www.pawoon.com/</code></td>\n<td>\n<p>- POS </p>\n<p>- CRM</p>\n</td>\n<td>\n<p>-Mobile app<i> </i></p>\n<p><i>- S</i>ubscription </p>\n<p>- Hardware </p>\n<p>- Digital Payment </p>\n</td>\n<td>- No banking partner</td>\n<td>\n<p>- e-Wallet (digital payment) </p>\n<p>- QRIS *MDR is applied </p>\n<p>- VA for subscriber</p>\n</td>\n<td>Prefered 100.000+ install</td>\n</tr>\n<tr>\n<td>Majoo</td>\n<td> </td>\n<td>\n<p>- POS - Digital Lending: (Majoo Capital) </p>\n<p>- Ads </p>\n<p>- Omnichannel</p>\n</td>\n<td>\n<p>- Mobile app</p>\n<p><i> - S</i>ubscription</p>\n<p> - Hardware </p>\n<p>- Digital Payment </p>\n</td>\n<td>\n<p>- modalku </p>\n<p>- BRI </p>\n<p>- Commonwealth </p>\n<p>- BRI Ventures</p>\n</td>\n<td>\n<p>- e-Wallet (digital payment) </p>\n<p>- QRIS *MDR is applied</p>\n</td>\n<td> </td>\n</tr>\n<tr>\n<td>Qasir</td>\n<td>qasir.id</td>\n<td>\n<p>- POS</p>\n<p>- Digital Lending</p>\n</td>\n<td>\n<p>- Mobile app<i> </i></p>\n<p><i>- S</i>ubscription </p>\n<p>- Hardware </p>\n<p>- Digital Payment </p>\n</td>\n<td>No banking partner</td>\n<td>\n<p>- e-Wallet (digital payment) </p>\n<p>- QRIS *MDR is applied</p>\n</td>\n<td>\n<p>Prefered but syariah</p>\n<p>500.000+ install</p>\n</td>\n</tr>\n<tr>\n<td> Olsera</td>\n<td> olsera.com</td>\n<td>\n<p> - POS </p>\n<p>- Omnichannel</p>\n</td>\n<td>\n<p> - Mobile app<i> </i></p>\n<p><i>- S</i>ubscription </p>\n<p>- Hardware </p>\n<p>- Digital Payment </p>\n</td>\n<td> No banking partner</td>\n<td>\n<p> - e-Wallet (digital payment)</p>\n<p> - QRIS *MDR is applied</p>\n</td>\n<td>Prefered 50.000+ Install</td>\n</tr>\n<tr>\n<td> Deal POS</td>\n<td> dealpos.co.id</td>\n<td>\n<p> - POS </p>\n<p>- Digital Lending </p>\n<p>- omnichannel </p>\n<p>- Payment link</p>\n</td>\n<td>\n<p> - Mobile apps </p>\n<p>- Hardware </p>\n<p>- subscription </p>\n<p>- Digital Payment </p>\n<p>- Omnichannel</p>\n</td>\n<td> No banking partner</td>\n<td>\n<p> - e-wallet(digital payment) </p>\n<p>- QRIS </p>\n<p>- Bank transfer/ATM</p>\n</td>\n<td>Prefered 5000+ Install</td>\n</tr>\n<tr>\n<td> cashlez</td>\n<td> </td>\n<td> </td>\n<td> </td>\n<td> </td>\n<td> </td>\n<td>10000+ install</td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<h3><a href=\"#recommendation\"></a> Recommendation</h3>\n<ul>\n<li>Moka (kontak: venny xBL)</li>\n<li>Pawoon (kontak: venny xBL)</li>\n<li>Majoo</li>\n<li>Olsera</li>\n<li>DealPos</li>\n<li>Qasir??? (syariah)</li>\n</ul>\n",
        "tanggal_content": "2022-12-28T10:07:55.290Z",
        "author_content": "Arum",
        "direktorat_content": "Tidak ada direktorat",
        "PIC_Analyst": "Tidak ada PIC Analyst",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/StartupKasirPOSResearch"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#analisis-transaksi-scan-qris-atau-pay-now-pada-mobile-banking-oleh-nasabah-bank-artha-graha-internasional\"></a> <strong>Analisis Transaksi Scan QRIS atau Pay Now pada Mobile Banking Oleh Nasabah Bank Artha Graha Internasional</strong></h1>",
        "text_content": "<p><span><strong>Analisis Transaksi Scan QRIS atau Pay Now pada Mobile Banking Oleh Nasabah Bank Artha Graha Internasional</strong></span></p>\n<p> </p>\n<figure>\n<table>\n<tbody>\n<tr>\n<th>Direktorat</th>\n<td> Digital Insight</td>\n</tr>\n<tr>\n<th>Sub Direktorat</th>\n<td> MIS</td>\n</tr>\n<tr>\n<th>Analysis Date</th>\n<td> 2 - 14 Agustus 2023</td>\n</tr>\n<tr>\n<th><strong>Analysis status</strong></th>\n<td> Done</td>\n</tr>\n<tr>\n<th><strong>Document status </strong></th>\n<td> Done</td>\n</tr>\n<tr>\n<th><strong>PIC  Analyst</strong></th>\n<td> Theresia, Ratih, Iqbal</td>\n</tr>\n<tr>\n<th>Stakeholder</th>\n<td> -</td>\n</tr>\n<tr>\n<th>Reference Document</th>\n<td>\n<ul>\n<li> </li>\n</ul>\n</td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h2><a href=\"#version-history\"></a> Version History</h2>\n<figure>\n<table>\n<tbody>\n<tr>\n<td><i><strong>Version</strong></i></td>\n<td><i><strong>Date</strong></i></td>\n<td><i><strong>Description</strong></i></td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n<tr>\n<td> </td>\n<td> </td>\n<td> </td>\n</tr>\n</tbody>\n</table>\n</figure>\n<p> </p>\n<p> </p>\n<p> </p>\n<p> </p>\n<h1><a href=\"#background\"></a> <strong>Background</strong></h1>\n<p>Seiring berkembangnya zaman, saat ini sudah banyak <i>merchant-merchant </i>ataupun penjual yang menggunakan metode <i>cashless</i> sebagai metode pembayaran, salah satunya dengan menggunakan QRIS. BAGI melalui aplikasi agi mobile menyediakan layanan <i>scan</i> QRIS atau Pay Now sebagai salah satu fasilitas untuk melakukan pembayaran secara <i>cashless</i> di <i>merchant-mechant </i>yang menyediakan faislitas tersebut. Diketahui transaksi penggunaan fasilitas <i>scan</i> QRIS atau Pay Now nasabah BAGI masih rendah. Untuk itu diambil sampel data transaksi <i>scan</i> QRIS atau Pay Now nasabah selama 6 bulan terakhir, yaitu pada periode Februari sampai dengan Juli 2023.</p>\n<h1><a href=\"#data-source-time-scope\"></a> <strong>Data Source &amp; Time Scope</strong></h1>\n<ol>\n<li>Data nasabah yang melakukan transaksi QRIS atau Pay Now pada Februari-Juli 2023</li>\n<li>Data CASA 31 Juli 2023</li>\n</ol>\n<h1><a href=\"#key-findings\"></a> <strong>Key Findings</strong></h1>\n<ol>\n<li>Dari 230.289 pengguna AGI Mobile, jumlah transaksi nasabah yang menggunakan <i>scan</i> QRIS/Pay Now hanya <strong>247.897 transaksi</strong> atau sebanding dengan 1x transaksi/user all time.</li>\n<li>Terdapat <strong>10.929 nasabah</strong> melakukan transaksi <i>scan</i> QRIS all time dan <strong>8.264 nasabah </strong>dalam periode Februari sampai dengan Juli 2023.</li>\n<li><strong>1.501 nasabah baru 2023 </strong>melakukan transaksi <i>scan</i> QRIS atau setidaknya 18% dari total nasabah yang bertransaksi QRIS pada Februari - Juli 2023.</li>\n<li><strong>Nasabah Artha Payroll </strong>merupakan nasabah yang paling banyak melakukan transaksi <i>scan </i>QRIS.</li>\n</ol>\n<h1><a href=\"#recommendation\"></a> <strong>Recommendation</strong></h1>\n<p>Sebutkan rekomendasi apa aja yang ingin dilakukan berdasarkan Key Findings</p>\n<h1><a href=\"#analysis\"></a> <strong>Analysis</strong></h1>\n<p>Tuliskan proses kalian dalam menganalisis, proses bisa diceritakan dahulu apa yang mau dicari, kemudian divisualisasi, lalu disimpulkan.</p>\n<h1><a href=\"#appendix\"></a> <strong>Appendix</strong></h1>\n<p>Masukin ke sini query, atau file2 kalian dari hasil analisis.</p>\n",
        "tanggal_content": "2023-08-04T07:57:55.310Z",
        "author_content": "Ratih",
        "direktorat_content": "Digital Insight",
        "PIC_Analyst": "Theresia, Ratih, Iqbal",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/templatedataanalysis"
    },
    {
        "topic_document": "Digital Insight",
        "title_content": "<h1><a href=\"#tracker-program-cashback\"></a> Tracker Program Cashback</h1>",
        "text_content": "<h1><a href=\"#tracker-program-cashback\"></a> Tracker Program Cashback</h1>\n<p> </p>\n<p>Cashback Rule</p>\n<ol>\n<li>Cashback diberikan untuk TRANSAKSI PERTAMA setiap customer SEJAK START PERIODE PROMO<ol>\n<li>merchantnya hanya ke yang eligible</li>\n<li>kalo customer transaksi ke 3 merchant yang eligible ambil yang pertama</li>\n</ol>\n</li>\n<li>Cashback hanya applied ke UMI, UKE</li>\n<li>Cashback hanya X% max RpY</li>\n</ol>\n<p>WHAT DO WE WANT TO KNOW?</p>\n<ol>\n<li>jumlah merchant yang eligible</li>\n<li>jumlah customer yang mendapat cashback</li>\n<li>jumlah transaksi yang mendapatkan cashback &gt; harusnya sama kaya point 2</li>\n<li>jumlah cashback yang keluar berdasarkan no 3 dnegan perhitungan % amount cashback</li>\n</ol>\n<p>Res:</p>\n<p>Query:</p>\n<p><a href=\"/automation_cashback_report.sql\">/automation_cashback_report.sql</a></p>\n<p> </p>\n<p> </p>\n",
        "tanggal_content": "2022-12-29T02:17:28.911Z",
        "author_content": "Arum",
        "direktorat_content": "Tidak ada direktorat",
        "PIC_Analyst": "Tidak ada PIC Analyst",
        "link_content": "http://10.0.232.117:3000/en/Insight/Data_Insight/TrackerProgramCashback"
    }
]
for test in data_test:
    pattern = r'\[.*?\]\s+(.*?)</h1>'
    test['text_content'] = re.sub(r'<img\s+src="', r'<img src="http://127.0.0.1:5000/get-image-semantic', test['text_content'])

    # Menghapus karakter \n pada atribut text_content
    test['text_content'] = test['text_content'].replace('\n', '')

    soup = BeautifulSoup(test['title_content'], 'html.parser')
    h1_element = soup.h1
    if h1_element:
        result = h1_element.get_text(strip=True)
        test['title_content'] = result
    else:
        print("Tidak ada ditemukan tag h1 didalam element:", test['title_content'])

    test["text_content"] = re.sub(r'<figure>.*?initial document.*?<\/figure>', '', test["text_content"], flags=re.DOTALL)
    test["text_content"] = re.sub(r'<figure>.*?Direktorat.*?<\/figure>', '', test["text_content"], flags=re.DOTALL)
    test["text_content"] = re.sub(r'<figure>.*?PIC Analyst.*?<\/figure>', '', test["text_content"], flags=re.DOTALL)    
    test["text_content"] = re.sub(r'<figure>.*?Description.*?<\/figure>', '', test["text_content"], flags=re.DOTALL)    

    test['text_content'] = remove_xa0_tags(test['text_content'])

    test['text_content'] = remove_h2_with_text(test['text_content'], "Version History")

    soup_2 = BeautifulSoup(test['text_content'], 'html.parser')
    p_elements = soup_2.find_all('p', string=re.compile(r'Analisis Transaksi Scan QRIS atau Pay Now pada Mobile Banking Oleh Nasabah Bank Artha Graha Internasional', re.IGNORECASE))
    # Ubah tag <p> menjadi <h1>
    for p_element in p_elements:
        h1_tag = soup_2.new_tag('h1')
        h1_tag.string = p_element.get_text()
        p_element.replace_with(h1_tag)

        # Update konten teks kembali ke data_test
        test['text_content'] = str(soup_2)
        
    new_data_test.append(test)



In [38]:
print(new_data_test)

[{'topic_document': 'Digital Insight', 'title_content': '[07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS', 'text_content': '<h1><a href="#h-07042022-analisis-target-akuisisi-dan-aktivasi-merchant-qris"></a> [07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS</h1><h2><a href="#latar-belakang-dan-tujuan"></a> Latar Belakang dan Tujuan</h2><p>Analisis ini dibuat sebagai dasar pembuatan dan desain kontrol untuk strategi akuisisi dan aktivasi merchant QRIS.</p><p>Target full year 2022 adalah akuisisi 100.000 merchant QRIS, dan 1 Trilyun CASA. Dimana transaksi QRIS adalah salah satu channel untuk mencapai target CASA. Oleh karena itu perlu dilakukan analisis untuk menyusun strategi akuisisi dan aktivasi merchant QRIS, menurunkan teknis pelaksanaan di lapangan, dan desain &amp; mekanisme kontrol pelaksanaannya.</p><h2><a href="#temuan-utama"></a> Temuan Utama</h2><h3><a href="#simulasi-target"></a> Simulasi Target</h3><p>Berangkat dari target 100 ribu customer dan as

In [39]:
for new_test in new_data_test:
    display(HTML(new_test['text_content']))

No,Promo,Bentuk Promo,Mekanisme Promo,Achievement
0,promo apa yang diusulkan?,"bentuknya diskon, cashback, atau apa?",bagiamana skema promo-nya?,apa yang mission yang diachieve untuk kita
1,Merchant discount,Discount or cashback,User mendapatkan sejumlah X% discount max A jika belanja di merchant tertentu (yang sudah ditentukan merchantnya) dengan AGI mobile/agicash,Meningkatkan activation dan retention user; target Activation Zone dan Hall of Fame
2,Mission Referral,Cashback & Gamification,Referrer akan mendapatkan cashback sejumlah X jika TELAH mencapai misi refer P orang untuk memuat akun agi mobile/agicash,Meningkatkan acquisition; Dengan gamification user akan distimulasi untuk melakukan misi mencapai target cashback; target Acquisition Zone
3,Mission Referral Transaction,Cashback & Gamification,Referrer dan Referee mendapatkan cashback sejumlah X jika Referee telah membuat transaksi pertama,Meningkatkan acquisition dan activation; Dengan gamification user akan distimulasi untuk melakukan misi mencapai target cashback; target Acquisition Zone dan Activation Zone
4,Mission Transaction,Cashback & Gamification,User mendapatkan cashback sejumlah X jika telah mencapai misi membuat transaksi sebanyak N kali,Meningkatkan activation; Dengan gamification user akan distimulasi untuk melakukan misi mencapai target cashback; target Activation Zone
5,Mission Create AGI Account,Cashback & Gamification,User mendapatkan cashback sejumlah X jika mendaftarkan akun menjadi nasabah AGI,Meningkatkan activation AGI acccount (Tabungan Icon bukan ya???)
6,Discount admin fee connect to emoney,Discount,"User mendapatkan discount admin fee untuk topup emoney; misalnya GOPAY, OVO (*lumayan 1000)",Meningkatkan activation dan retention user; target Activation Zone dan Hall of Fame
7,Voucher redeem,Cashback,"AGI menyediakan sejulah voucher untuk di redeem, user bisa kapan saja redeem; voucher juga memiliki waktu expired",Meningkatkan activation dan retention user; target Activation Zone dan Hall of Fame
8,Donasi,-,Menambahkan fitur donasi ketika melakukan pembayaran,Meningkatkan value “for others”
9,"Program mendapatkan ide dari internal, sebulan sekali atau 2 bulan sekali",Voucher atau cashback di Agi mobile,"Membuat program untuk intenal AGI (ditentukan siapa); siapa pun yang punya ide untuk meningkatkan acquisition atau activation atau program yang bsa membantu mencapai target, maka akan mendaatkan voucher yang bisa di redeem",Meningkatkan bounding di internal AGI dan gotong royong mencapat target; sekaligus mengajak karyawan menggunakan agi mobile dan bertransaksi dengan agi mobile


Kuadran,Count of ACCOUNT_NUMBER
I,2
II,16
III,3146
IV,15
Grand Total,3179
Kuadran,Count of ACCOUNT_NUMBER
I,6
II,67
III,3062
IV,44


No,Korban,Pelaku,BAG
1,,Menghubungi Korban mengaku sebagai karyawan Shopee,
2,Dihubungi Pelaku bahwa Korban mendapat hadiah/cashback senilai (contoh Rp. 2jt),,
3,,Meminta Korban download aplikasi Pinjaman Onliner (ex: Kredivo),
4,Mendowload dan registrasi akun Pinjaman nline,,
5,,Meminta informasi data Pinjol Korban - OTP - KTP - Foto - Ibu Kandung - Email - dll.,
6,Memberikan informasi data Pinjol ke Pelaku,,
7,,Membajak akun Pinjol Korban,
8,,Membuat trasnaksi pinjaman di aku Pinjol Korban,
9,Mendapat Notifikasi Pinjaman dari aplikasi Pinjol,,
10,,Membuat akun BAG atas nama Korban,


Startup,Website,Tag,Bisnis Model,Partnership,QRIS/VA/Payment Channel Use Case,
Moka,https://www.mokapos.com/,- POS - CRM - Digital Lending (Moka Capital),- Tablet is installed Moka POS mobile app - Hardware - Subscription - Moka Capital (digital lending) - Digital Payment,"- PT Mandiri Capital Indonesia, Bank Mandiri - Modalku - Commonwealth",- e-Wallet (digital payment) - QRIS *MDR is applied,
Pawoon,https://www.pawoon.com/,- POS - CRM,-Mobile app - Subscription - Hardware - Digital Payment,- No banking partner,- e-Wallet (digital payment) - QRIS *MDR is applied - VA for subscriber,Prefered 100.000+ install
Majoo,,- POS - Digital Lending: (Majoo Capital) - Ads - Omnichannel,- Mobile app - Subscription - Hardware - Digital Payment,- modalku - BRI - Commonwealth - BRI Ventures,- e-Wallet (digital payment) - QRIS *MDR is applied,
Qasir,qasir.id,- POS- Digital Lending,- Mobile app - Subscription - Hardware - Digital Payment,No banking partner,- e-Wallet (digital payment) - QRIS *MDR is applied,Prefered but syariah500.000+ install
Olsera,olsera.com,- POS - Omnichannel,- Mobile app - Subscription - Hardware - Digital Payment,No banking partner,- e-Wallet (digital payment) - QRIS *MDR is applied,Prefered 50.000+ Install
Deal POS,dealpos.co.id,- POS - Digital Lending - omnichannel - Payment link,- Mobile apps - Hardware - subscription - Digital Payment - Omnichannel,No banking partner,- e-wallet(digital payment) - QRIS - Bank transfer/ATM,Prefered 5000+ Install
cashlez,,,,,,10000+ install


In [17]:
documents = []
embeddings = []
metadatas = []
ids = []

for index, data in enumerate(new_data_test):
    documents.append(data['text_content'])
    embedding = model.encode(data['text_content']).tolist()
    embeddings.append(embedding)
    metadatas.append({'title': data['title_content'], 'topic_document' : data['topic_document'], 'tanggal_content' : data['tanggal_content'],
                      'author_content' : data['author_content'], 'link_content': data['link_content'], 'direktorat_content' : data['direktorat_content'],
                     'pic_analyst_content' : data['PIC_Analyst']})
    ids.append(str(index + 1))

In [18]:
import chromadb
client = chromadb.Client()
agi_collection_emb = client.create_collection("agi_collection_emb")

In [19]:
agi_collection_emb.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

In [20]:
query = "Simulai kondisi realistis pada dokumen analisis target akuisisi dan aktivasi merchant".lower()
input_em = model.encode(query).tolist()

results = agi_collection_emb.query(
    query_embeddings=[input_em],
    n_results=3,
)

In [21]:
output = []
for metadata, document_list, distances, ids in zip(results['metadatas'], results['documents'], results['distances'], results['ids']):
    for metadata_item, document, distance, ids_doc in zip(metadata, document_list, distances, ids):
        title = metadata_item['title']
        topic_document = metadata_item['topic_document']
        tanggal_content = metadata_item['tanggal_content']
        author_content = metadata_item['author_content']
        link_content = metadata_item['link_content']
        direktorat_content =  metadata_item['direktorat_content']
        pic_analyst_content = metadata_item['pic_analyst_content']
        output.append({
            'ids_doc' : ids_doc,
            'title': title,
            'document': document,
            'topic_document' :topic_document,
            'tanggal_content' : tanggal_content,
            'author_content' : author_content,
            'direktorat_content' : direktorat_content,
            'pic_analyst_content' : pic_analyst_content,
            'link_content' : link_content,
            'length' : len(document),
            'distance_doc' : distance,
        })

In [22]:
for doc in output:
    print(doc['title'])

[07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS
Analisis Kuadran Merchant berdasarkan Transaksi, Balance, dan Amount
[08042022] Database Promo Ideation


In [23]:
### tuning model

In [24]:
# model_sentence_list = ["msmarco-MiniLM-L6-cos-v5",
#                       "msmarco-MiniLM-L12-cos-v5",
#                       "msmarco-distilbert-cos-v5",
#                       "multi-qa-MiniLM-L6-cos-v1",
#                       "multi-qa-distilbert-cos-v1",
#                       "multi-qa-mpnet-base-cos-v1",]

# for model_item in model_sentence_list:
#     modelBaca = Reducer(model_item)

#     answer = []
#     for doc in output:
#         answerNew= modelBaca.reduce(doc['document'], query, num_sents=4)
#         answer.append({"id_doc" : doc['ids_doc'],"title_doc" : doc['title'], "content_doc" : answerNew, "length_doc" : len(answerNew), "distance_doc" : round(doc['distance_doc'], 3) })



# # print(answer)    



In [25]:
modelBaca = Reducer("msmarco-MiniLM-L-6-v3")

answer = []
for doc in output:
    answerNew= modelBaca.reduce(doc['document'], query, num_sents=6)
    content = answerNew
    soup = BeautifulSoup(content, 'html.parser')
    formatted_content = soup.prettify()
    
    # Mengembalikan konten yang telah diperbaiki ke dalam objek data
    answer.append({"id_doc" : doc['ids_doc'],"title_doc" : doc['title'], "content_doc" : formatted_content, "length_doc" : len(answerNew), "distance_doc" : round(doc['distance_doc'], 3),
                  "topic_documents" : doc['topic_document'], "tanggal_content" : doc['tanggal_content'], "author_content" : doc['author_content'],
                  "link_content" : doc['link_content'], 'direktorat_content' : doc['direktorat_content'], 'pic_analyst_content' : doc['pic_analyst_content']})

# print(answer)

In [26]:
answer[0:3]

[{'id_doc': '1',
  'title_doc': '[07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS',
  'content_doc': '<h1>\n <a href="#h-07042022-analisis-target-akuisisi-dan-aktivasi-merchant-qris">\n </a>\n [07042022] Analisis Target Akuisisi dan Aktivasi Merchant QRIS\n</h1>\n<h2>\n <a href="#latar-belakang-dan-tujuan">\n </a>\n Latar Belakang dan Tujuan\n</h2>\n<p>\n Analisis ini dibuat sebagai dasar pembuatan dan desain kontrol untuk strategi akuisisi dan aktivasi merchant QRIS.\n</p>\n<p>\n Target full year 2022 adalah akuisisi 100.000 merchant QRIS, dan 1 Trilyun CASA. Oleh karena itu perlu dilakukan analisis untuk menyusun strategi akuisisi dan aktivasi merchant QRIS, menurunkan teknis pelaksanaan di lapangan, dan desain &amp; mekanisme kontrol pelaksanaannya.\n</p>\n<h2>\n <a href="#temuan-utama">\n </a>\n Temuan Utama\n</h2>\n<h3>\n <a href="#simulasi-target">\n </a>\n Simulasi Target\n</h3>\n<p>\n Berangkat dari target 100 ribu customer dan asumsi 1T target CASA bisa didapatka

In [45]:
# Use any translator you like, in this example GoogleTranslator
translatedGPT = GoogleTranslator(source='id', target='en').translate("rangkum kalimat berikut :" + """Latar Belakang dan Tujuan Analisis ini dibuat sebagai dasar pembuatan dan desain kontrol untuk strategi akuisisi dan aktivasi merchant QRIS. Target full year 2022 adalah akuisisi 100.000 merchant QRIS, dan 1 Trilyun CASA. Dimana transaksi QRIS adalah salah satu channel untuk mencapai target CASA. Oleh karena itu perlu dilakukan analisis untuk menyusun strategi akuisisi dan aktivasi merchant QRIS, menurunkan teknis pelaksanaan di lapangan, dan desain & mekanisme kontrol pelaksanaannya. Temuan Utama Simulasi Target Berangkat dari target 100 ribu customer dan asumsi 1T target CASA bisa didapatkan seluruhnya dari transaksi QRIS. Skema simulasi menggunakan skema Referral yang sudah terdefinisi di dokumen ‘Target 1 Juta Nasabah’. Simulasi kondisi ideal Dalam simulasi ini, metric yang harus dijaga untuk dapat mencapai tujuan adalah: Metric akuisisi merchant: Rasio Merchant Aktif: 20% Merchant MGM % Merchant Aktif: 30% Rata-rata Akuisisi per Merchant MGM: 5 merchant Metric aktivasi merchant: Banyaknya Transaksi per Merchant: 150 transaksi Rata-rata Nilai Transaksi: IDR 100.000 """ + " menjadi kalimat yang rapi dan benar")

In [46]:
modelGPT = GPT4All("ggml-model-gpt4all-falcon-q4_0.bin")

Found model file at  /home/diginsight/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin
falcon_model_load: loading model from '/home/diginsight/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: ........................ done
falcon_model_load: model size =  3872.59 MB / num tensors = 196


In [47]:
print(translatedGPT)

summarize the following sentences :Background and Purpose This analysis is made as a basis for the creation and design of controls for QRIS merchant acquisition and activation strategy. The full year 2022 target is the acquisition of 100,000 QRIS merchants and 1 trillion CASA. Where QRIS transactions are one of the channels to achieve CASA targets. Therefore it is necessary to carry out an analysis to develop a strategy for acquiring and activating QRIS merchants, reducing technical implementation in the field, and designing & controlling mechanisms for its implementation. Main Findings of Target Simulation Departing from a target of 100 thousand customers and assuming 1T CASA targets can be obtained entirely from QRIS transactions. The simulation scheme uses the Referral scheme that has been defined in the 'Target of 1 Million Customers' document. Simulation of ideal conditions In this simulation, the metrics that must be maintained to achieve the goal are: Merchant acquisition metric

In [48]:
output_3 = modelGPT.generate(translatedGPT)

In [51]:
output_3 = GoogleTranslator(source='en', target='id').translate(output_3)

In [52]:
display(HTML(output_3))

In [ ]:
for data_answer_2 in answer:
    print(data_answer_2['distance_doc'])

In [65]:
answer_4 = []

for data_answer_4 in answer:
    soup_2 = BeautifulSoup(data_answer_4['content_doc'], 'html.parser')
    h1_tags = soup_2.find_all('h1')
    h2_tags = soup_2.find_all('h2')
    h3_tags = soup_2.find_all('h3')
    p_tags = soup_2.find_all('p')
    li_tags = soup_2.find_all('li')
    table_tags = soup_2.find_all('table')

    for h1_tag in h1_tags:
        h1_tag['class'] = 'text-base text-[#41B4BE] font-bold'
    for h2_tag in h2_tags:
        h2_tag['class'] = 'text-xs text-[#41B4BE] font-semibold'
    for h3_tag in h3_tags:
        h3_tag['class'] = 'text-xs text-[#41B4BE] font-semibold'
    for p_tag in p_tags:
        p_tag['class'] = 'text-xs text-[#41B4BE] font-regular'
    for li_tag in li_tags:
        li_tag['class'] = 'text-xs text-red-500'
    for table_tag in table_tags:
        table_tag['class'] = 'table-auto border border-collapse border-blue-500'

    # Tambahkan kode untuk mengganti div dengan tag p
    div_tags = soup_2.find_all('div', class_='item-content-document')
    for div_tag in div_tags:
        if div_tag.get_text(strip=True):
            new_p_tag = soup_2.new_tag('p', class_='text-xs text-[#41B4BE] font-regular')
            new_p_tag.string = div_tag.get_text()
            div_tag.replace_with(new_p_tag)

    data_answer_4['content_doc'] = str(soup_2.prettify())
    answer_4.append(data_answer_4)

In [66]:
num = 0
for data_answer in answer_4:
    num = num + 1
    display(HTML(f"Isi Value Query : {query}</h1>"))
    display(HTML(f"<h1>Data Summary ke-{(num)}</h1>"))
    display(HTML(f"<h1>===================================================</h1>"))
    display(HTML(data_answer['content_doc']))

In [60]:
answer_2 = []

for in_answer in answer:
    answer_2.append(in_answer)

In [ ]:
new_answer_2 = []
for doc1 in answer_2:
    matching_data2 = next((doc2 for doc2 in data_test_2 if doc2["title_content"] == doc1["title_doc"]), None)
    if matching_data2:
        doc1["doc_link_content"] = matching_data2["link_content"]
        new_answer_2.append(doc1)

In [ ]:
print(new_answer_2)